# 스프린트 AI3 초급 프로젝트
## 1팀(약탐정단) - 공지연

In [ ]:
# kaggle install
!pip install kaggle

In [ ]:
# 베이스라인 YOLOv8 설치
!pip install ultralytics

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.3.1+cu121
Uninstalling torch-2.3.1+cu121:
  Successfully uninstalled torch-2.3.1+cu121
Found existing installation: torchvision 0.18.1+cu121
Uninstalling torchvision-0.18.1+cu121:
  Successfully uninstalled torchvision-0.18.1+cu121
Found existing installation: torchaudio 2.3.1+cu121
Uninstalling torchaudio-2.3.1+cu121:
  Successfully uninstalled torchaudio-2.3.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.3.1%2Bcu121-cp311-cp311-linux_x86_64.whl (781.0 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.18.1%2Bcu121-cp311-cp311-linux_x86_64.whl (7.0 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.3.1%2Bcu121-cp311-cp311-linux_x86_64.whl (3.4 MB)


In [ ]:
# @title 라이브러리 관리

# 데이터 관련
import os
import json
import random
import cv2
import yaml # YAML 파일 생성
import shutil
from itertools import combinations
import shutil

# 평가
import re

# YOLO
from ultralytics import YOLO


# 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image

# 데이터 다루기
from sklearn.model_selection import train_test_split

# 기타 유틸리티
from tqdm import tqdm
from google.colab import drive

In [ ]:
# @title 폰트 설치 및 셋팅
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
!ls -l /usr/share/fonts/truetype/nanum/

total 33300
-rw-r--r-- 1 root root 4415956 May 14  2020 NanumBarunGothicBold.ttf
-rw-r--r-- 1 root root 4186060 May 14  2020 NanumBarunGothic.ttf
-rw-r--r-- 1 root root 4642244 May 14  2020 NanumGothicBold.ttf
-rw-r--r-- 1 root root 2246240 May 14  2020 NanumGothicCodingBold.ttf
-rw-r--r-- 1 root root 2315924 May 14  2020 NanumGothicCoding.ttf
-rw-r--r-- 1 root root 4691820 May 14  2020 NanumGothic.ttf
-rw-r--r-- 1 root root 4183592 May 14  2020 NanumMyeongjoBold.ttf
-rw-r--r-- 1 root root 3839464 May 14  2020 NanumMyeongjo.ttf
-rw-r--r-- 1 root root  733500 May 14  2020 NanumSquareB.ttf
-rw-r--r-- 1 root root 1030948 May 14  2020 NanumSquareRoundB.ttf
-rw-r--r-- 1 root root 1063276 May 14  2020 NanumSquareRoundR.ttf
-rw-r--r-- 1 root root  723640 May 14  2020 NanumSquareR.ttf


In [ ]:
# 폰트 오류 해결의 최최최종본..🥹

# 1. 사용할 폰트 파일의 정확한 경로 지정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 2. Matplotlib의 폰트 매니저에 해당 폰트 파일 경로를 직접 추가
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)

    # 3. 폰트 파일의 이름을 알아내어 전역 폰트로 설정
    font_name = fm.FontProperties(fname=font_path).get_name()
    plt.rc('font', family=font_name)
    plt.rc('axes', unicode_minus=False)
    print(f"✅ Matplotlib 폰트 매니저에 '{font_name}' 폰트를 직접 등록하고, 전역 기본값으로 설정하였습니다.")
else:
    print(f" 지정된 경로에 폰트 파일이 없습니다: {font_path}")

# 4. Ultralytics가 사용하는 기본 폰트 파일도 교체
ultralytics_font_path = '/root/.config/Ultralytics/Arial.ttf'
if os.path.exists(font_path):
    os.system(f"cp '{font_path}' '{ultralytics_font_path}'")
    print(f"✅ Ultralytics의 기본 폰트(Arial.ttf)를 '{font_name}'으로 교체했습니다.")

✅ Matplotlib 폰트 매니저에 'NanumGothic' 폰트를 직접 등록하고, 전역 기본값으로 설정하였습니다.
✅ Ultralytics의 기본 폰트(Arial.ttf)를 'NanumGothic'으로 교체했습니다.


In [ ]:
font_title = fm.FontProperties(fname=font_path, size=18)
font_label = fm.FontProperties(fname=font_path, size=12)
font_ticks = fm.FontProperties(fname=font_path, size=10)

In [ ]:
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# kaggle.json 위치 변경 및 실행
# json 파일(토큰)을 업로드하고 실행시켜야 함
# 캐글 -> Settings -> API 토큰 발급

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# @title 데이터 다운로드
# 실행 전 캐글 참가 규칙에 동의해야함. --> 안그러면 에러
!kaggle competitions download -c ai03-level1-project

100% 3.90G/3.91G [01:48<00:00, 40.1MB/s]
100% 3.91G/3.91G [01:48<00:00, 38.7MB/s]


In [ ]:
# 파일 압축해제
!unzip -q ai03-level1-project.zip -d ./data

# 데이터 다운로드 - 구글 드라이브 연동

In [ ]:
print("구글 드라이브를 연결합니다...")
drive.mount('/content/drive')

구글 드라이브를 연결합니다...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file_path = '/content/drive/MyDrive/스프린트프로젝트/data/pill_images_v3.zip'

In [ ]:
print(f"확인 대상 파일 경로: {zip_file_path}")

확인 대상 파일 경로: /content/drive/MyDrive/스프린트프로젝트/data/pill_images_v3.zip


In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   53G   61G  47% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.2G  775M  61% /usr/sbin/docker-init
/dev/sda1        74G   58G   17G  78% /opt/bin/.nvidia
tmpfs           6.4G  2.8M  6.4G   1% /var/colab
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           113G   56G   58G  49% /content/drive


In [ ]:
print("\n파일을 Colab 로컬로 복사합니다. 잠시만 기다려주세요...")
!cp '/content/drive/MyDrive/스프린트프로젝트/data/pill_images_v3.zip' /content/pill_images_v3.zip
print("✅ 로컬 복사 완료.")


파일을 Colab 로컬로 복사합니다. 잠시만 기다려주세요...
✅ 로컬 복사 완료.


In [ ]:
# -o 옵션을 추가하여 묻지 않고 모든 파일을 덮어쓰도록 합니다.
!unzip -o /content/pill_images_v3.zip -d /content/pill_data_v3

In [ ]:
print("\n최종 폴더 구조를 확인합니다...")
!ls -R /content/pill_data_v3

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
K-003483-019861-025367-035206_0_2_0_2_75_000_200.json

/content/pill_data_v3/raw_data/train_annotations/K-003483-019861-025367-036637_json:
K-003483  K-019861  K-025367  K-036637

/content/pill_data_v3/raw_data/train_annotations/K-003483-019861-025367-036637_json/K-003483:
K-003483-019861-025367-036637_0_2_0_2_70_000_200.json
K-003483-019861-025367-036637_0_2_0_2_75_000_200.json
K-003483-019861-025367-036637_0_2_0_2_90_000_200.json

/content/pill_data_v3/raw_data/train_annotations/K-003483-019861-025367-036637_json/K-019861:
K-003483-019861-025367-036637_0_2_0_2_75_000_200.json
K-003483-019861-025367-036637_0_2_0_2_90_000_200.json

/content/pill_data_v3/raw_data/train_annotations/K-003483-019861-025367-036637_json/K-025367:
K-003483-019861-025367-036637_0_2_0_2_70_000_200.json
K-003483-019861-025367-036637_0_2_0_2_75_000_200.json
K-003483-019861-025367-036637_0_2_0_2_90_000_200.json

/content/pill_data_v3/raw_data/train_annotations/K-003483-019861-02

# 데이터 확인 및 탐색

In [ ]:
# 경로 지정
data_dir = './data'

In [ ]:
# 해당 경로와 그 하위 폴더 및 파일 목록 출력
# 너무 많은 출력을 방지하기 위해 일부만 확인함.
for dirpath, dirnames, filenames in os.walk(data_dir):
    print(f'폴더 경로: {dirpath}')
    print(f'폴더 이름: {dirnames}')
    print(f'파일 : {filenames[:5]}')
    print("-" * 10)

폴더 경로: ./data
폴더 이름: ['train_annotations', 'train_images', 'test_images']
파일 : []
----------
폴더 경로: ./data/train_annotations
폴더 이름: ['K-003483-022347-027653-029667_json', 'K-002483-005886-019552-025438_json', 'K-002483-004378-012778-025438_json', 'K-003483-019861-029667-031885_json', 'K-003351-018357-022074_json', 'K-003351-031863-035206_json', 'K-003351-018147-029667_json', 'K-003483-027733-029667-031885_json', 'K-001900-010224-016551-031705_json', 'K-003483-027653-029667-030308_json', 'K-003483-020238-025367-027733_json', 'K-002483-005094-005886-006192_json', 'K-003544-010221-016551-029345_json', 'K-003483-016232-027777-034597_json', 'K-003483-016232-027733-030308_json', 'K-003544-004543-016551-029871_json', 'K-003351-016262-020014_json', 'K-003483-016232-025367-025469_json', 'K-001900-016548-029345-033208_json', 'K-003351-035206-041768_json', 'K-003483-020877-022347-036637_json', 'K-003483-016262-019861-030308_json', 'K-003351-033880-035206_json', 'K-002483-005094-012081-022627_json

각 이미지마다 별도의 JSON 파일이 존재하며, 매우 복잡한 폴더 구조 안에 흩어져 있음.

큰 json 파일 안에 데이터가 담겨있을 것이라 예상했는데.. 아주 복잡한 결과 (당황 ;_;)

### < 경로 예시 >
- 폴더 경로: ./data/train_annotations/K-003351-033880-038162_json/K-033880
- 파일: ['K-003351-033880-038162_0_2_0_2_70_000_200.json']


### EDA 계획

1. train_annotations 폴더와 그 모든 하위 폴더를 탐색
2. 발견된 모든 json 파일 탐색
3. 이미지는 PNG 형식, 주석은 COCO 포맷(JSON)형식인 것을 최대한 활용해서 데이터를 살펴보자


# EDA

In [ ]:
# train_annotations 폴더 경로
annotations_dir = './data/train_annotations/'

all_images = []
all_annotations = []
all_categories = []
category_ids = set() # 카테고리 중복 추가를 방지하기 위한 집합

print("모든 JSON 파일 통합 과정 진행 ~ ")

# annotations_dir와 그 모든 하위 폴더를 탐색
for dirpath, dirnames, filenames in tqdm(os.walk(annotations_dir)):
    for filename in filenames:
        if filename.endswith('.json'):
            json_path = os.path.join(dirpath, filename)

            with open(json_path, 'r') as f:
                data = json.load(f)

                # 각 파일의 image, annotation 정보 추가
                if 'images' in data:
                    all_images.extend(data['images'])
                if 'annotations' in data:
                    all_annotations.extend(data['annotations'])

                # 카테고리 정보는 중복 없이 추가
                if 'categories' in data:
                    for category in data['categories']:
                        if category['id'] not in category_ids:
                            all_categories.append(category)
                            category_ids.add(category['id'])

print("파일 통합 완료!")

# 하나로 합쳐진 COCO 데이터 생성
coco_data = {
    'images': all_images,
    'annotations': all_annotations,
    'categories': all_categories
}

모든 JSON 파일 통합 과정 진행 ~ 


2375it [00:01, 1864.87it/s]

파일 통합 완료!


In [ ]:
# 통합된 데이터의 전체 개수 확인

# 다루기 쉬운 coco를 이용해 coco 통합 데이터 생성
print(f"총 이미지 개수: {len(coco_data['images'])}")
print(f"총 어노테이션 개수: {len(coco_data['annotations'])}")
print(f"총 카테고리 개수: {len(coco_data['categories'])}")
print("-" * 20)

# Pandas DataFrame으로 변환하여 샘플 확인
images_df = pd.DataFrame(coco_data['images'])
annotations_df = pd.DataFrame(coco_data['annotations'])
categories_df = pd.DataFrame(coco_data['categories'])

print("--- 통합된 images_df 샘플 ---")
display(images_df.head())

print("\n--- 통합된 annotations_df 샘플 ---")
display(annotations_df.head())

print("\n--- 통합된 categories_df 샘플 ---")
display(categories_df.head())

총 이미지 개수: 4526
총 어노테이션 개수: 4526
총 카테고리 개수: 73
--------------------
--- 통합된 images_df 샘플 ---


file_name  width  height  \
0  K-003483-022347-027653-029667_0_2_0_2_70_000_2...    976    1280   
1  K-003483-022347-027653-029667_0_2_0_2_90_000_2...    976    1280   
2  K-003483-022347-027653-029667_0_2_0_2_75_000_2...    976    1280   
3  K-003483-022347-027653-029667_0_2_0_2_70_000_2...    976    1280   
4  K-003483-022347-027653-029667_0_2_0_2_90_000_2...    976    1280   

                                             imgfile    drug_N drug_S  \
0  K-003483-022347-027653-029667_0_2_0_2_70_000_2...  K-003483   정상알약   
1  K-003483-022347-027653-029667_0_2_0_2_90_000_2...  K-003483   정상알약   
2  K-003483-022347-027653-029667_0_2_0_2_75_000_2...  K-003483   정상알약   
3  K-003483-022347-027653-029667_0_2_0_2_70_000_2...  K-027653   정상알약   
4  K-003483-022347-027653-029667_0_2_0_2_90_000_2...  K-027653   정상알약   

  back_color drug_dir light_color  camera_la  ...  img_regist_ts  \
0     연회색 배경       앞면         주백색         70  ...       20041222   
1     연회색 배경       앞면         주백색         90  ...       20041222   
2     연회색 배경       앞면         주백색         75  ...       20041222   
3     연회색 배경       앞면         주백색         70  ...       20100810   
4     연회색 배경       앞면         주백색         90  ...       20100810   

   form_code_name mark_code_front_anal mark_code_back_anal  \
0           필름코팅정                                            
1           필름코팅정                                            
2           필름코팅정                                            
3           필름코팅정                                            
4           필름코팅정                                            

  mark_code_front_img mark_code_back_img mark_code_front mark_code_back  \
0                                                                         
1                                                                         
2                                                                         
3                                                                         
4                                                                         

  change_date    id  
0    20200723  1366  
1    20200723  1368  
2    20200723  1367  
3    20210331  1366  
4    20210331  1368  

[5 rows x 50 columns]


--- 통합된 annotations_df 샘플 ---


area  iscrowd                  bbox  category_id  ignore segmentation  \
0  48198        0  [132, 287, 277, 174]         3482       0           []   
1  48140        0  [583, 845, 290, 166]         3482       0           []   
2  50112        0  [558, 812, 288, 174]         3482       0           []   
3  44600        0  [670, 256, 223, 200]        27652       0           []   
4  45732        0   [94, 853, 222, 206]        27652       0           []   

     id  image_id  
0  5205      1366  
1  5206      1368  
2  5207      1367  
3  5200      1366  
4  5199      1368


--- 통합된 categories_df 샘플 ---


supercategory     id                     name
0          pill   3482       기넥신에프정(은행엽엑스)(수출용)
1          pill  27652             세비카정 10/40mg
2          pill  22346               자누비아정 50mg
3          pill  29666                 리바로정 4mg
4          pill   5885  타이레놀이알서방정(아세트아미노펜)(수출용)

In [ ]:
unique_image_count = images_df['file_name'].nunique()
print(f"총 이미지 개수: {unique_image_count}")

총 이미지 개수: 1489


## 1. 데이터 구조
- 총 이미지 개수: 4526
- 총 어노테이션 개수: 4526

--> 이미지의 개수와 어노테이션 개수가 정확히 일치하는 것을 보아, 하나의 Json 파일이 알약 하나에 대한 정보만 담고 있다는 것을 시사할 수 있을 것 같습니다.

## 2. 중복된 이미지 존재
- ***images_df*** 의 샘플(특히, 0번째행과 1번째행과 4번째행)을 살펴보면, 서로 다른행임에도 불구하고 파일 이름이 동일한 경우가 있는 것 같습니다. 즉, 행의 개수가 4526개지만, 실제 이미지 파일의 개수가 아니라 **전체 알약(어노테이션)의 개수**임을 파악할 수 있었습니다.

    ☑️ 한 이미지에 알약이 3개 있다면, 해당 이미지 정보가 3번 중복 기록된 것을 알 수 있습니다.

## 3. 데이터의 양과 특징
- images_df에는 이미지 파일 이름 외에도 drug_N(알약 코드), drug_S(알약 상태), light_color(조명 색) 등 무려 50개에 달하는 상세한 메타데이터가 포함된 것으로 보입니다. 데이터 증강을 도입했을 때의 특정 조건(예: 어두운 조명)에서 학습할 때 유용할 수 있을 것 같습니다.



In [ ]:
# 어노테이션에서 카테고리별 개수 세기
category_counts = annotations_df['category_id'].value_counts().reset_index()
category_counts.columns = ['id', 'count']

# 카테고리 이름 정보와 합쳐서 'category_df_counts' 생성
category_df_counts = pd.merge(categories_df, category_counts, on='id')
category_df_counts = category_df_counts.sort_values(by='count', ascending=False)

In [ ]:
# @title 1️⃣ 알약 클래스는 얼마나 균일하게 분포해 있는가?

plt.figure(figsize=(12, 10))
sns.barplot(x='count', y='name', data=category_df_counts.head(20))

# fontproperties' 인자를 통해 폰트 객체 직접 전달
plt.title('상위 20개 클래스 분포', fontproperties=font_title)
plt.xlabel('어노테이션 개수', fontproperties=font_label)
plt.ylabel('클래스 이름', fontproperties=font_label)

# get_xticklabels()는 숫자 눈금이라 괜찮을 수 있지만, 만일을 위해 함께 설정함
plt.xticks(fontproperties=font_ticks)
plt.yticks(fontproperties=font_ticks)


plt.show()

## 🔎 클래스(알약) 분포 결과

### 1. 심각한 클래스 불균형
- *'가네신정(은행엽엑스)(수출용)'* 클래스가 500개가 넘는 압도적으로 많은 데이터를 갖고 있습니다.
- 20위인 *'카나브정 60mg'*은 100개에도 미치지 못하고 있습니다.
    - 위 그래프는 상위 20개의 분석 결과물이므로, 나머지 53개 클래스는 이보다 데이터가 훨씬 더 적을 것으로 예상합니다.

### 2. 롱테일 분포(Long-tail Distribution) 형태
- 데이터가 롱테일 분포 형태를 띄고있어 편향되거나, 낮은 일반화 성능이 나올 수 있습니다.



In [ ]:
# @title 2️⃣ 이미지 크기 분포
print("--- 이미지 크기 기술 통계 ---")
print(images_df[['width', 'height']].describe())

# 산점도로 시각화
plt.figure(figsize=(8, 6))
sns.scatterplot(x='width', y='height', data=images_df)

# 수정된 부분: fontproperties 직접 주입
plt.title('이미지 크기 분포', fontproperties=font_title)
plt.xlabel('너비 (width)', fontproperties=font_label)
plt.ylabel('높이 (height)', fontproperties=font_label)
plt.xticks(fontproperties=font_ticks)
plt.yticks(fontproperties=font_ticks)

plt.show()

## 🔎 이미지 크기 분포 분석 결과
| | width | height |
|:---|:---|:---|
| count | 4526.0 | 4526.0 |
| mean | 976.0 | 1280.0 |
| std | 0.0 | 0.0 |
| min | 976.0 | 1280.0 |
| 25% | 976.0 | 1280.0 |
| 50% | 976.0 | 1280.0 |
| 75% | 976.0 | 1280.0 |
| max | 976.0 | 1280.0 |

### 모든 이미지 크기가 동일합니다. 🥹🫶
- 모든 이미지가 가로 976 픽셀, 세로 1280픽셀로 완전히 동일합니다.
    - 근거 1. 시각화 한 산점도 그래프에 모든 데이터가 하나의 점으로 찍혀있다.
        - 모든 데이터의 (x,y)값이 같다는 의미로 해석할 수 있다.
    - 근거 2. 기술 통계표에 따르면, std값이 width와 height 모두 0.0이다.
        - std가 0인 것은 데이터에 어떠한 변동도 없이 모든 값이 평균값과 동일하다는 근거가 될 수 있다.

--> 이를 통해 모든 이미지의 리사이즈 및 패딩 과정을 생략할 수 있음을 알 수 있습니다. 또한, 안정적인 모델 학습을 기대해볼 수 있습니다.

In [ ]:
# @title 3️⃣ 이미지 당 객체(알약)수 분포
# 이미지당 어노테이션 개수 세기
annotations_per_image = annotations_df['image_id'].value_counts()

# 히스토그램으로 시각화
plt.figure(figsize=(10, 6))
sns.histplot(annotations_per_image, bins=30, kde=False)

# 수정된 부분: fontproperties 직접 주입
plt.title('이미지 당 객체(알약) 수 분포', fontproperties=font_title)
plt.xlabel('한 이미지 내의 알약 개수', fontproperties=font_label)
plt.ylabel('이미지 수', fontproperties=font_label)
plt.xticks(fontproperties=font_ticks)
plt.yticks(fontproperties=font_ticks)

plt.show()

## 🔎 이미지 당 객체 수 분포 결과

1. 한 이미지에 여러 알약이 포함되어있다.
    - 그래프를 통해 이미지 한 장에 알약이 3개 포함된 경우가 600장 이상입니다.
        - 4개, 2개인 경우도 다수 존재하며, 알약이 1개만 있는 이미지는 상대적으로 적습니다.

2. 이미지 정보 중복 -> 확정
    - **images_df**와 함께 이미지 정보 중복의 원인을 확정할 수 있습니다.
        -  알약 4개가 들어있는 이미지 한 장은 4개의 개별 JSON 파일로 나뉘어 있으며, 제가 데이터를 합치는 과정에서 images_df와 annotations_df에 각각 4개의 행이 생성된 것으로 보입니다.

--> 이미지 한 장을 불러올 때마다 해당 이미지에 속한 모든 어노테이션(바운딩 박스)들을 함께 그룹화해서 모델에 전달해주어야함.


In [ ]:
# @title 4️⃣ 바운딩 박스 크기 분포

# 바운딩 박스 면적 계산 (이전에 실행했다면 생략 가능)
if 'bbox_area' not in annotations_df.columns:
    annotations_df['bbox_area'] = annotations_df['bbox'].apply(lambda x: x[2] * x[3])

# 면적 분포 시각화 (너무 큰 값은 제외하고 확인)
plt.figure(figsize=(10, 6))
sns.histplot(annotations_df[annotations_df['bbox_area'] < 100000]['bbox_area'], bins=50)

# 수정된 부분: fontproperties 직접 주입
plt.title('바운딩 박스 크기(면적) 분포', fontproperties=font_title)
plt.xlabel('면적 (width * height)', fontproperties=font_label)
plt.ylabel('개수', fontproperties=font_label)
plt.xticks(fontproperties=font_ticks)
plt.yticks(fontproperties=font_ticks)

plt.show()

## 🔎 바운딩 박스 크기 분포 결과

- 다봉형 분포 형태
    - 면적이 30,000~ 60,000 사이에 큰 그룹이 형성되어 있고, 80,000 근처에도 또 다른 작은 그룹이 존재합니다.


- 다봉형 분포 형태 추측

    1.   실제 알약 크기의 차이 --> 실제로 알약의 크기는 매우 다양하다.(색깔, 크기 모두)
    2.   촬영 방향 및 형태의 차이 --> 같은 알약이라도 정면에서 찍었을 때와 측면에서 찍었을 때 등과 같은 상황에서 바운딩 박스의 면적이 달라질 수 있다.



In [ ]:
# @title 5️⃣ Visual Sanity Check
def visualize_sample(images_df, annotations_df, categories_df, num_samples=None, sample_files=None):
    """
    데이터셋에서 이미지를 시각화하는 함수.
    num_samples: 무작위로 보여줄 샘플 개수
    sample_files: 시각화할 특정 파일 이름의 리스트
    """
    # num_samples와 sample_files 중 하나만 사용하도록 처리
    if num_samples is not None:
        unique_image_files = images_df['file_name'].unique()
        files_to_show = random.sample(list(unique_image_files), num_samples)
    elif sample_files is not None:
        files_to_show = sample_files
    else:
        print("에러 - 파일 확인 요망")
        return

    # 선택된 파일들에 대해 시각화 진행
    for file_name in files_to_show:
        image_info_rows = images_df[images_df['file_name'] == file_name]
        if image_info_rows.empty:
            print(f"에러: {file_name}에 대한 이미지 정보를 찾을 수 없습니다.")
            continue

        image_id = image_info_rows.iloc[0]['id']
        img_path = os.path.join('./data/train_images', file_name)

        # 해당 이미지에 속한 모든 어노테이션 가져오기
        ann_infos = annotations_df[annotations_df['image_id'] == image_id]

        # 이미지 열기
        img = Image.open(img_path).convert('RGB')
        fig, ax = plt.subplots(1, figsize=(10, 10))
        ax.imshow(img)
        ax.axis('off')

        # 바운딩 박스 및 라벨 그리기
        for _, ann in ann_infos.iterrows():
            bbox = ann['bbox']
            cat_id = ann['category_id']
            cat_name = categories_df[categories_df['id'] == cat_id]['name'].values[0]

            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],
                                     linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)

            ax.text(bbox[0], bbox[1] - 10, cat_name,
                    color='white', backgroundcolor='lime',
                    fontproperties=fm.FontProperties(fname=font_path, size=12))

        ax.set_title(f"Image: {file_name}", fontproperties=font_title)
        plt.show()

# 3개의 샘플 이미지 시각화
visualize_sample(images_df, annotations_df, categories_df, num_samples=3)

## 🔎 Visual Sanity Check 결과

1. 라벨링 품질: 매우 우수
2. 이미지가 균일한 조명 아래에서 촬영되었음을 알 수 있고, 알약의 형태와 문자가 선명하게 보입니다. 겹치는 알약이 없는 것 같습니다.
3. 각 알약은 색상(붉은색, 흰색, 녹색 등)과 표면에 새겨진 고유한 문자/기호('T' 로고, 'ATV', 'C77', 'G40')를 가지고 있습니다. 명확한 클래스 구분 특징이 존재합니다.
4. 일부 누락된 라벨링을 발견했습니다.

In [ ]:
# @title 6️⃣ 메타데이터 탐색
"""
1. drug_dir(알약 방향) --> 알약의 앞면과 뒷면 데이터가 균등하게 분포하는가?
2. light_color (조명 색)--> 특정 조명 환경에서 촬영된 이미지가 편중되어 있는가?
"""
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. 알약 방향 분포
sns.countplot(x='drug_dir', data=images_df, ax=axes[0])
axes[0].set_title('알약 방향(drug_dir) 분포', fontproperties=font_title)
axes[0].set_xlabel('방향', fontproperties=font_label)
axes[0].set_ylabel('개수', fontproperties=font_label)

# 2. 조명 색 분포
sns.countplot(x='light_color', data=images_df, ax=axes[1])
axes[1].set_title('조명 색(light_color) 분포', fontproperties=font_title)
axes[1].set_xlabel('조명', fontproperties=font_label)
axes[1].set_ylabel('개수', fontproperties=font_label)

# 각 라벨에 폰트 적용
for ax in axes:
    for label in ax.get_xticklabels():
        label.set_fontproperties(font_ticks)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font_ticks)

plt.tight_layout()
plt.show()


## 🔎 메타데이터 분석 결과

1. 데이터의 압도적인 앞면 편향이 있습니다.
2. 조명 색 분포 그래프가 매우 극단적입니다. 모든 데이터가 주백색 이라는 단 하나의 조명 조건에서 촬영되었음을 알 수 있습니다.


### 메타데이터 시사점

1. 혹시나 만약에, 뒷면으로 촬영되거나 뒷면을 인식해야하는 일이 생긴다면 그런 능력을 가질 가능성은 사실상 0에 가까워 보입니다. 데이터셋의 한계라고 봅니다.
2. 모든 데이터가 주백색 조명에서 촬영되었기 때문에 학습이 쉬워질 수 있습니다.

    2-1. 하지만 모델이 주백색 조명에만 익숙해지기 때문에 실생활 데이터에 모델을 노출시킨다면 성능이 급격히 저하될 가능성이 매우 커 보입니다. 이는 앞으로 '데이트 증강' 적용이 꼭 필요하다는 증거가 될 것 같습니다.

In [ ]:
# @title 7️⃣ 바운딩 박스의 형태(가로세로 비율) 분석

# 가로세로 비율(aspect ratio) 계산
annotations_df['bbox_aspect_ratio'] = annotations_df['bbox'].apply(lambda x: x[2] / x[3])

# 히스토그램으로 시각화
plt.figure(figsize=(10, 6))
sns.histplot(annotations_df['bbox_aspect_ratio'], bins=50)

plt.title('바운딩 박스 가로/세로 비율 분포', fontproperties=font_title)
plt.xlabel('가로/세로 비율 (width / height)', fontproperties=font_label)
plt.ylabel('개수', fontproperties=font_label)
plt.xticks(fontproperties=font_ticks)
plt.yticks(fontproperties=font_ticks)
plt.show()

## 🔎 바운딩 박스 형태 분석 결과

1. 대부분의 알약이 정원형에 가까운 형태입니다. 가로세로 비율(aspect ratio) 그래프를 보면 1.0 주변에 매우 높고 뾰족한 봉우리가 있습니다. 이는 우리 데이터셋에 있는 대부분의 알약이 가로와 세로 길이가 거의 같은, 즉 정원형 또는 정사각형에 가까운 형태를 가지고 있음을 의미합니다.
2. 1.0을 중심으로 양옆으로도 데이터가 분포하고 있습니다. 이는 캡슐처럼 한쪽으로 길거나(비율 > 1.0), 세로로 길쭉한(비율 < 1.0) 형태의 알약도 일부 포함되어 있음을 보여줍니다. 하지만 그 수는 원형 알약에 비해 현저히 적은 것으로 보입니다.


### 시사점
우리가 학습할 모델은 가로세로 비율이 1.0에 가까운 앵커 박스들을 중심으로 설정할 때, 대부분의 알약을 훨씬 더 효율적이고 정확하게 찾아낼 가능성이 높다는 점을 시사할 수 있습니다.

In [ ]:
# @title 8️⃣ 극단적인 데이터 확인하기
# 이름이나 모양이 비슷한 약들끼리 서로 라벨이 잘못 붙은 경우가 있을까?

# 1. 가장 면적이 '작은' 어노테이션 3개 정보 찾기
smallest_bbox_anns = annotations_df.sort_values(by='bbox_area').head(3)
print("--- 1️⃣ 가장 작은 바운딩 박스 샘플 ---")
display(smallest_bbox_anns)

# 2. 가장 면적이 '큰' 어노테이션 3개 정보 찾기
largest_bbox_anns = annotations_df.sort_values(by='bbox_area', ascending=False).head(3)
print("\n--- 2️⃣ 가장 큰 바운딩 박스 샘플 ---")
display(largest_bbox_anns)

# 3. 가장 '길쭉한' (비율이 큰) 어노테이션 3개 정보 찾기
aspect_ratio_anns = annotations_df.sort_values(by='bbox_aspect_ratio', ascending=False).head(3)
print("\n--- 3️⃣ 가장 길쭉한 바운딩 박스 샘플 ---")
display(aspect_ratio_anns)

# 위에서 찾은 샘플 중 하나를 직접 시각화 (예: 가장 작은 샘플 첫 번째)
sample_to_visualize = images_df[images_df['id'] == smallest_bbox_anns.iloc[0]['image_id']]
if not sample_to_visualize.empty:
    visualize_sample(images_df, annotations_df, categories_df,
                     sample_files=[sample_to_visualize.iloc[0]['file_name']])

--- 1️⃣ 가장 작은 바운딩 박스 샘플 ---


area  iscrowd                  bbox  category_id  ignore segmentation  \
2268  18492        0  [209, 314, 134, 138]        23202       0           []   
3626  18626        0  [215, 290, 134, 139]        23202       0           []   
1965  18760        0  [210, 314, 134, 140]        23202       0           []   

       id  image_id  bbox_area  bbox_aspect_ratio  
2268  700       184      18492           0.971014  
3626  918       245      18626           0.964029  
1965  612       160      18760           0.957143


--- 2️⃣ 가장 큰 바운딩 박스 샘플 ---


area  iscrowd                 bbox  category_id  ignore segmentation  \
3693  272435        0  [17, 651, 529, 515]        16687       0           []   
4056  270384        0  [19, 649, 516, 524]        16687       0           []   
3601  265500        0  [33, 520, 450, 590]        16687       0           []   

        id  image_id  bbox_area  bbox_aspect_ratio  
3693   207        57     272435           1.027184  
4056  1425       381     270384           0.984733  
3601  5520      1454     265500           0.762712


--- 3️⃣ 가장 길쭉한 바운딩 박스 샘플 ---


area  iscrowd                  bbox  category_id  ignore segmentation  \
195   86884        0  [502, 848, 428, 203]        35205       0           []   
1093  53106        0  [523, 889, 334, 159]        27992       0           []   
2007  53600        0  [522, 888, 335, 160]        27992       0           []   

        id  image_id  bbox_area  bbox_aspect_ratio  
195    958       255      86884           2.108374  
1093  3032       798      53106           2.100629  
2007  4312      1133      53600           2.093750

In [ ]:
# @title 8️⃣-1 클래스 교차 확인

def visualize_by_category(category_name, num_samples=3):
    print(f"--- '{category_name}' 클래스 샘플 시각화 ---")
    try:
        # 카테고리 이름으로 ID 찾기
        cat_id = categories_df[categories_df['name'].str.contains(category_name)].iloc[0]['id']

        # 해당 카테고리 ID를 가진 어노테이션 찾기
        ann_ids = annotations_df[annotations_df['category_id'] == cat_id]['image_id']

        # 해당 이미지 ID를 가진 이미지 정보 찾기
        img_infos = images_df[images_df['id'].isin(ann_ids)]

        # 랜덤 샘플링
        sample_files = img_infos['file_name'].sample(num_samples)

        # 시각화 함수 호출
        visualize_sample(images_df, annotations_df, categories_df, sample_files=sample_files)

    except IndexError:
        print(f"'{category_name}'을 포함하는 클래스를 찾을 수 없습니다.")

# 이름이 비슷한 '리피토정' 샘플들 확인
visualize_by_category('리피토정', num_samples=2)

# 이름이 비슷한 '자누메트' 샘플들 확인
visualize_by_category('자누메트', num_samples=2)

In [ ]:
# @title 알약의 극단값만 모아보기
# 1. 가장 면적이 '작은' 어노테이션 3개 정보 찾기
smallest_bbox_anns = annotations_df.sort_values(by='bbox_area').head(3)
print("--- 가장 작은 바운딩 박스 샘플 ---")
display(smallest_bbox_anns)

# 2. 가장 면적이 '큰' 어노테이션 3개 정보 찾기
largest_bbox_anns = annotations_df.sort_values(by='bbox_area', ascending=False).head(3)
print("\n--- 가장 큰 바운딩 박스 샘플 ---")
display(largest_bbox_anns)

# 3. 가장 '길쭉한' (비율이 큰) 어노테이션 3개 정보 찾기
aspect_ratio_anns = annotations_df.sort_values(by='bbox_aspect_ratio', ascending=False).head(3)
print("\n--- 가장 길쭉한 바운딩 박스 샘플 ---")
display(aspect_ratio_anns)

# 위에서 찾은 샘플 중 하나를 직접 시각화 (예: 가장 작은 샘플 첫 번째)
sample_to_visualize = images_df[images_df['id'] == smallest_bbox_anns.iloc[0]['image_id']]
if not sample_to_visualize.empty:
    visualize_sample(images_df, annotations_df, categories_df,
                     sample_files=[sample_to_visualize.iloc[0]['file_name']])

## 🔎 극단적 데이터 확인 결과

- 샘플을 확인한 결과, '가바토파정 100mg'과 같은 라벨링 자체에는 큰 이상 없습니다. 라벨링 자체는 매우 우수합니다.
- 바운딩 박스도 정확합니다.
- 클래스도 교차된 부분 없는 것으로 확인됩니다.

### 📌 주목할 점

1. 결과 테이블을 보면, 가장 작은 바운딩 박스 3개는 모두 동일한 **category_id (23202)**를 가지고 있고, 가장 큰 박스 3개 역시 모두 동일한 **category_id (16687)**를 가집니다.
  - 이는 극단적인 크기의 바운딩 박스가 라벨링 실수가 아니라, **'원래 아주 작은 알약'**과 **'원래 아주 큰 알약'**이라는 해당 클래스들의 고유한 물리적 특징임을 시사합니다. '큐에타핀정 25mg'이 실제로 다른 약에 비해 작다는 것을 확인할 수 있습니다.

2. 캡슐 형태 알약이 여러개 존재하는 것이 증명되었습니다.
  - 가로/세로 비율이 2.0을 넘는 박스들은 너비가 높이의 2배 이상인, 옆으로 길쭉한 형태를 가집니다. 다양한 형태의 객체 모두 학습하는데 집중해야함을 알 수 있습니다.



In [ ]:
# 이전에 정의한 visualize_sample 함수를 여러 번 실행
print("\n --- 무작위 샘플 1 --- \n ")
visualize_sample(images_df, annotations_df, categories_df, num_samples=1)

print("--- 무작위 샘플 2 --- \n ")
visualize_sample(images_df, annotations_df, categories_df, num_samples=1)

print("--- 무작위 샘플 3 --- \n ")
visualize_sample(images_df, annotations_df, categories_df, num_samples=1)

# 데이터 전처리

## 베이스라인 모델 선정 : YOLO
- 본 프로젝트는 주어진 경구약제 이미지에서 약물의 종류와 위치를 탐지하는 모델을 개발하는 것을 목표로 합니다. 이미지는 PNG 형식, 주석은 **COCO 포맷(JSON)** 으로 제공되어있는 데이터의 특징을 고려하여 속도와 정확도의 균형이 뛰어난 YOLO 아키텍처를 베이스라인 모델로 선정했습니다.
- 이에 맞는 데이터 전처리 과정을 거쳐 3개의 ***YOLO 모델(v10n, v8n, v8s)*** 에 대한 베이스라인 성능 평가를 진행하겠습니다.

In [ ]:
# @title 필수 전처리 과정
# 전체 고유 이미지 파일 이름 목록을 가져옴.
unique_images = images_df['file_name'].unique()
print(f"전체 고유 이미지 수: {len(unique_images)}")

# 이미지 목록을 훈련용과 검증용으로 8:2 비율로 나눔
train_files, val_files = train_test_split(unique_images, test_size=0.2, random_state=42)

print(f"훈련용 이미지 수: {len(train_files)}")
print(f"검증용 이미지 수: {len(val_files)}")

# 결과 확인 (샘플 5개씩 출력)
print("\n--- 훈련용 이미지 샘플 --- \n")
print(train_files[:5])

print("\n--- 검증용 이미지 샘플 ---\n ")
print(val_files[:5])

전체 고유 이미지 수: 1489
훈련용 이미지 수: 1191
검증용 이미지 수: 298

--- 훈련용 이미지 샘플 --- 

['K-003544-004543-010221-016548_0_2_0_2_75_000_200.png' 'K-003483-027653-029667-034597_0_2_0_2_75_000_200.png' 'K-003483-016262-027733-034597_0_2_0_2_70_000_200.png' 'K-003351-020238-041768_0_2_0_2_70_000_200.png' 'K-003483-016262-027733-034597_0_2_0_2_75_000_200.png']

--- 검증용 이미지 샘플 ---
 
['K-003351-016262-018147_0_2_0_2_90_000_200.png' 'K-003483-020877-034597-036637_0_2_0_2_90_000_200.png' 'K-001900-016548-033009-033208_0_2_0_2_70_000_200.png' 'K-003544-010221-016548-033009_0_2_0_2_70_000_200.png' 'K-003483-025469-031885-035206_0_2_0_2_75_000_200.png']


In [ ]:
os.makedirs('./data/train/labels', exist_ok=True)
os.makedirs('./data/val/labels', exist_ok=True)

In [ ]:
# 클래스 ID 매핑 생성
# 카테고리 DataFrame을 id 기준으로 정렬합니다.
sorted_categories = categories_df.sort_values(by='id').reset_index(drop=True)

# 원본 category_id를 새로운 0-indexed ID로 매핑하는 딕셔너리 생성
category_id_map = {row['id']: index for index, row in sorted_categories.iterrows()}

print("✅ 클래스 ID 맵 생성 완료.")


✅ 클래스 ID 맵 생성 완료.


In [ ]:
def convert_to_yolo_format_final(file_list, source_image_df, source_ann_df, target_label_dir, id_map):

    for file_name in tqdm(file_list, desc=f"'{target_label_dir}' 생성 중"):
        image_info_rows = source_image_df[source_image_df['file_name'] == file_name]
        if image_info_rows.empty:
            continue

        image_info = image_info_rows.iloc[0]
        img_w, img_h, img_id = image_info['width'], image_info['height'], image_info['id']

        ann_infos = source_ann_df[source_ann_df['image_id'] == img_id]

        label_file_path = os.path.join(target_label_dir, file_name.replace('.png', '.txt'))
        with open(label_file_path, 'w') as f:
            for _, ann in ann_infos.iterrows():
                original_cat_id = ann['category_id']
                if original_cat_id in id_map:
                    yolo_class_id = id_map[original_cat_id]
                else:
                    continue

                x_min, y_min, w, h = ann['bbox']
                x_max = min(x_min + w, img_w)
                y_max = min(y_min + h, img_h)
                x_min = max(x_min, 0)
                y_min = max(y_min, 0)
                w_clipped = x_max - x_min
                h_clipped = y_max - y_min

                x_center = (x_min + w_clipped / 2) / img_w
                y_center = (y_min + h_clipped / 2) / img_h
                width_norm = w_clipped / img_w
                height_norm = h_clipped / img_h

                f.write(f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}\n")

# --- 라벨 파일 재생성 실행 ---
train_labels_dir = './data/train/labels'
val_labels_dir = './data/val/labels'

os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

convert_to_yolo_format_final(train_files, images_df, annotations_df, train_labels_dir, category_id_map)
convert_to_yolo_format_final(val_files, images_df, annotations_df, val_labels_dir, category_id_map)

print("\n✅ 모든 라벨 파일 변환 및 저장이 완료되었습니다.")

'./data/val/labels' 생성 중: 100%|██████████| 298/298 [00:00<00:00, 677.05it/s]


✅ 모든 라벨 파일 변환 및 저장이 완료되었습니다.


In [ ]:
# 라벨 파일이 저장된 폴더 경로
train_labels_dir = './data/train/labels'
val_labels_dir = './data/val/labels'

# 각 폴더의 파일 개수 세기
num_train_labels = len(os.listdir(train_labels_dir))
num_val_labels = len(os.listdir(val_labels_dir))

print(f"훈련용 이미지 수: {len(train_files)}")
print(f"생성된 훈련용 라벨 파일 수: {num_train_labels}")
print("-" * 30)
print(f"검증용 이미지 수: {len(val_files)}")
print(f"생성된 검증용 라벨 파일 수: {num_val_labels}")
print("-" * 30)

# 두 숫자가 일치하는지 최종 확인
if len(train_files) == num_train_labels and len(val_files) == num_val_labels:
    print("✅ 이미지 수와 라벨 파일 수가 정확히 일치합니다.")
else:
    print("❗️이미지 수와 라벨 파일 수가 일치하지 않습니다.")

훈련용 이미지 수: 1191
생성된 훈련용 라벨 파일 수: 1191
------------------------------
검증용 이미지 수: 298
생성된 검증용 라벨 파일 수: 298
------------------------------
✅ 이미지 수와 라벨 파일 수가 정확히 일치합니다.


In [ ]:
#클래스 이름 목록을 생성. (id 순서대로 정렬)
# YOLO는 class_id를 0부터 순서대로 인식하므로, id를 기준으로 정렬하는 것이 중요함.
class_names = categories_df.sort_values(by='id')['name'].tolist()

# os.path.abspath()로 현재 경로를 기준으로 절대 경로 지정
train_img_path = os.path.abspath('./data/train/images')
val_img_path = os.path.abspath('./data/val/images')

# YAML 파일에 쓸 데이터 구성 (절대 경로 사용)
data_yaml = {
    'train': train_img_path, # 훈련 이미지 폴더 경로
    'val': val_img_path, # 검증 이미지 폴더 경로
    'nc': len(class_names), # 클래스 개수
    'names': class_names # 클래스 이름 목록
}

# YAML 파일로 저장
yaml_path = './data/dataset.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, allow_unicode=True, default_flow_style=False)

print(f"✅ 데이터셋 설정 파일이 '{yaml_path}'에 저장되었습니다.")
print("\n--- 생성된 파일 내용 ---")
with open(yaml_path, 'r') as f:
    print(f.read())

✅ 데이터셋 설정 파일이 './data/dataset.yaml'에 저장되었습니다.

--- 생성된 파일 내용 ---
names:
- 보령부스파정 5mg
- 뮤테란캡슐 100mg
- 일양하이트린정 2mg
- 기넥신에프정(은행엽엑스)(수출용)
- 무코스타정(레바미피드)(비매품)
- 알드린정
- 뉴로메드정(옥시라세탐)
- 타이레놀정500mg
- 에어탈정(아세클로페낙)
- 삼남건조수산화알루미늄겔정
- 타이레놀이알서방정(아세트아미노펜)(수출용)
- 삐콤씨에프정 618.6mg/병
- 조인스정 200mg
- 쎄로켈정 100mg
- 리렉스펜정 300mg/PTP
- 아빌리파이정 10mg
- 자이프렉사정 2.5mg
- 다보타민큐정 10mg/병
- 써스펜8시간이알서방정 650mg
- 에빅사정(메만틴염산염)(비매품)
- 리피토정 20mg
- 크레스토정 20mg
- 가바토파정 100mg
- 동아가바펜틴정 800mg
- 오마코연질캡슐(오메가-3-산에틸에스테르90)
- 란스톤엘에프디티정 30mg
- 리리카캡슐 150mg
- 종근당글리아티린연질캡슐(콜린알포세레이트) 
- 콜리네이트연질캡슐 400mg
- 트루비타정 60mg/병
- 스토가정 10mg
- 노바스크정 5mg
- 마도파정
- 플라빅스정 75mg
- 엑스포지정 5/160mg
- 펠루비정(펠루비프로펜)
- 아토르바정 10mg
- 라비에트정 20mg
- 리피로우정 20mg
- 자누비아정 50mg
- 맥시부펜이알정 300mg
- 메가파워정 90mg/병
- 쿠에타핀정 25mg
- 비타비백정 100mg/병
- 놀텍정 10mg
- 자누메트정 50/850mg
- 큐시드정 31.5mg/PTP
- 아모잘탄정 5/100mg
- 세비카정 10/40mg
- 트윈스타정 40/5mg
- 카나브정 60mg
- 울트라셋이알서방정
- 졸로푸트정 100mg
- 트라젠타정(리나글립틴)
- 비모보정 500/20mg
- 레일라정
- 리바로정 4mg
- 렉사프로정 15mg
- 트라젠타듀오정 2.5/850mg
- 낙소졸정 500/20mg
- 아질렉트정(라사길린메실산염)
-

In [ ]:
# 원본 이미지가 있는 폴더
source_dir = './data/train_images'

# 복사할 대상 폴더 경로
train_target_dir = './data/train/images'
val_target_dir = './data/val/images'

# 대상 폴더 생성 (이미 존재하면 무시)
os.makedirs(train_target_dir, exist_ok=True)
os.makedirs(val_target_dir, exist_ok=True)

# 훈련용 이미지 파일 복사
print(f"'{train_target_dir}' 폴더로 훈련용 이미지 복사 중...")
for file_name in tqdm(train_files):
    source_path = os.path.join(source_dir, file_name)
    target_path = os.path.join(train_target_dir, file_name)
    shutil.copy(source_path, target_path)

# 검증용 이미지 파일 복사
print(f"\n'{val_target_dir}' 폴더로 검증용 이미지 복사를 중...")
for file_name in tqdm(val_files):
    source_path = os.path.join(source_dir, file_name)
    target_path = os.path.join(val_target_dir, file_name)
    shutil.copy(source_path, target_path)

print("\n✅ 모든 이미지 파일 복사가 완료되었습니다.")

'./data/train/images' 폴더로 훈련용 이미지 복사 중...


100%|██████████| 1191/1191 [00:17<00:00, 66.53it/s]



'./data/val/images' 폴더로 검증용 이미지 복사를 중...


100%|██████████| 298/298 [00:03<00:00, 75.19it/s]


✅ 모든 이미지 파일 복사가 완료되었습니다.


In [ ]:
# 훈련 결과 한글 폰트가 깨져서 예방 코드 추가
!cp /usr/share/fonts/truetype/nanum/NanumGothic.ttf /root/.config/Ultralytics/Arial.ttf

In [ ]:
# 사전 훈련된 YOLOv8n 모델 로드
model = YOLO('yolov8n.pt')

"""
# --- 모델 훈련 시작 ---
# data: 방금 완벽하게 만든 YAML 파일 경로
# epochs: 전체 데이터를 30번 반복 학습
# imgsz: 이미지를 640x640 크기로 조정하여 학습
# project/name: 결과가 저장될 폴더 이름
"""
results = model.train(
    data='./data/dataset.yaml',
    epochs=20,
    imgsz=640,
    project='runs/detect',
    name='yolov8n_baseline_e20(2)',
    exist_ok=True
)

print("\n✅ YOLOv8s 베이스라인 모델 훈련이 완료되었습니다.")
print("훈련 결과는 'runs/detect/yolov8n_baseline_e20(2)' 폴더에 저장되었습니다.")

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_baseline_e20(2), nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

train: Scanning /content/data/train/labels.cache... 1191 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]

train: /content/data/train/images/K-003351-016262-018357_0_2_0_2_75_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.8642]
train: /content/data/train/images/K-003544-004543-012247-016551_0_2_0_2_70_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.9723]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1090.4±1137.8 MB/s, size: 1760.1 KB)


val: Scanning /content/data/val/labels.cache... 298 images, 0 backgrounds, 0 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]


...

                   all        298        940      0.786      0.909      0.852      0.834

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20       2.8G     0.2421      1.038     0.8138         15        640: 100%|██████████| 75/75 [00:47<00:00,  1.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.37it/s]


                   all        298        940      0.795      0.901      0.847      0.833

20 epochs completed in 0.310 hours.
Optimizer stripped from runs/detect/yolov8n_baseline_e20(2)/weights/last.pt, 6.4MB
Optimizer stripped from runs/detect/yolov8n_baseline_e20(2)/weights/best.pt, 6.4MB

Validating runs/detect/yolov8n_baseline_e20(2)/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,091,487 parameters, 0 gradients, 8.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:08<00:00,  1.12it/s]


                   all        298        940      0.791      0.898      0.856      0.839
            보령부스파정 5mg         31         31      0.825          1      0.898       0.86
           뮤테란캡슐 100mg         31         31      0.752          1      0.881      0.805
           일양하이트린정 2mg         54         54      0.882          1      0.958      0.955
    기넥신에프정(은행엽엑스)(수출용)        109        109      0.817          1      0.859      0.853
     무코스타정(레바미피드)(비매품)         20         20      0.763          1      0.816      0.804
                  알드린정         18         18      0.866          1      0.895      0.889
          뉴로메드정(옥시라세탐)          9          9          1       0.93      0.995      0.961
            타이레놀정500mg          4          4      0.623      0.835      0.912      0.912
          에어탈정(아세클로페낙)          8          8       0.76          1      0.845      0.845
         삼남건조수산화알루미늄겔정          5          5      0.585          1      0.622      0.571
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# 2. YOLOv10n 모델 로드 및 훈련
model = YOLO('yolov10n.pt')

# 모델 훈련 시작
results = model.train(
    data='./data/dataset.yaml',
    epochs=20,
    imgsz=640,
    project='runs/detect',
    name='yolov10n_baseline_e20',
    exist_ok=True
)

print("\n✅ YOLOv10n 베이스라인 모델 훈련이 완료되었습니다.")
print("훈련 결과는 'runs/detect/yolov10n_baseline_e20' 폴더에서 확인하실 수 있습니다.")

100%|██████████| 5.59M/5.59M [00:00<00:00, 318MB/s]

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10n_baseline_e20, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

 23        [16, 19, 22]  1    910012  ultralytics.nn.modules.head.v10Detect        [73, [64, 128, 256]]          
YOLOv10n summary: 223 layers, 2,755,724 parameters, 2,755,708 gradients, 8.6 GFLOPs

Transferred 493/595 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3656.2±1640.3 MB/s, size: 1733.5 KB)


train: Scanning /content/data/train/labels.cache... 1191 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]

train: /content/data/train/images/K-003351-016262-018357_0_2_0_2_75_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.8642]
train: /content/data/train/images/K-003544-004543-012247-016551_0_2_0_2_70_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.9723]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      3.58G     0.4543      3.979      1.608         15        640: 100%|██████████| 75/75 [00:48<00:00,  1.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


                   all        298        940       0.66      0.766      0.776      0.767

20 epochs completed in 0.319 hours.
Optimizer stripped from runs/detect/yolov10n_baseline_e20/weights/last.pt, 5.8MB
Optimizer stripped from runs/detect/yolov10n_baseline_e20/weights/best.pt, 5.8MB

Validating runs/detect/yolov10n_baseline_e20/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10n summary (fused): 102 layers, 2,289,429 parameters, 0 gradients, 6.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


                   all        298        940      0.657      0.768      0.775      0.766
            보령부스파정 5mg         31         31      0.663      0.839      0.794      0.794
           뮤테란캡슐 100mg         31         31      0.645      0.935       0.73      0.678
           일양하이트린정 2mg         54         54      0.764      0.944      0.851      0.851
    기넥신에프정(은행엽엑스)(수출용)        109        109      0.717      0.991      0.841      0.841
     무코스타정(레바미피드)(비매품)         20         20      0.489      0.814      0.575      0.572
                  알드린정         18         18       0.78      0.593      0.776       0.77
          뉴로메드정(옥시라세탐)          9          9      0.563      0.778      0.753      0.753
            타이레놀정500mg          4          4      0.663          1      0.845      0.845
          에어탈정(아세클로페낙)          8          8      0.498      0.875      0.808       0.78
         삼남건조수산화알루미늄겔정          5          5      0.549      0.977       0.72       0.72
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# 'yolov8n.pt' 보다 한 단계 더 큰 'yolov8s.pt' 모델을 사용합니다.
model = YOLO('yolov8s.pt')

# 모델 훈련 시작
results = model.train(
    data='./data/dataset.yaml',
    epochs=20,
    imgsz=640,
    project='runs/detect',
    name='yolov8s_baseline_e20',
    exist_ok=True
)

print("\n✅ YOLOv8s 베이스라인 모델 훈련이 완료되었습니다.")
print("훈련 결과는 'runs/detect/yolov8s_baseline_e20' 폴더에서 확인하실 수 있습니다.")


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_baseline_e20, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

train: Scanning /content/data/train/labels.cache... 1191 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]

train: /content/data/train/images/K-003351-016262-018357_0_2_0_2_75_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.8642]
train: /content/data/train/images/K-003544-004543-012247-016551_0_2_0_2_70_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     3.9723]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.7±1.5 ms, read: 1554.7±1270.6 MB/s, size: 1760.1 KB)


val: Scanning /content/data/val/labels.cache... 298 images, 0 backgrounds, 0 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]


...

                   all        298        940      0.799      0.969      0.903      0.893

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      4.91G     0.2056     0.5153     0.7986         15        640: 100%|██████████| 75/75 [00:47<00:00,  1.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:06<00:00,  1.53it/s]


                   all        298        940      0.798      0.968      0.903      0.895

20 epochs completed in 0.314 hours.
Optimizer stripped from runs/detect/yolov8s_baseline_e20/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_baseline_e20/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_baseline_e20/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,153,835 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


                   all        298        940      0.799      0.968      0.904      0.896
            보령부스파정 5mg         31         31      0.835          1      0.911      0.896
           뮤테란캡슐 100mg         31         31      0.753          1      0.806      0.751
           일양하이트린정 2mg         54         54      0.881          1      0.908      0.908
    기넥신에프정(은행엽엑스)(수출용)        109        109      0.818          1      0.844      0.844
     무코스타정(레바미피드)(비매품)         20         20      0.758          1      0.797      0.797
                  알드린정         18         18      0.898          1      0.901      0.901
          뉴로메드정(옥시라세탐)          9          9      0.971          1      0.995      0.995
            타이레놀정500mg          4          4      0.752          1      0.945      0.945
          에어탈정(아세클로페낙)          8          8      0.755          1      0.939      0.939
         삼남건조수산화알루미늄겔정          5          5      0.605          1      0.818      0.818
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# 모델 결과 저장
!zip -r runs.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/yolov8n_baseline_e20(2)/ (stored 0%)
  adding: runs/detect/yolov8n_baseline_e20(2)/train_batch750.jpg (deflated 16%)
  adding: runs/detect/yolov8n_baseline_e20(2)/train_batch0.jpg (deflated 8%)
  adding: runs/detect/yolov8n_baseline_e20(2)/BoxR_curve.png (deflated 8%)
  adding: runs/detect/yolov8n_baseline_e20(2)/results.csv (deflated 59%)
  adding: runs/detect/yolov8n_baseline_e20(2)/labels.jpg (deflated 32%)
  adding: runs/detect/yolov8n_baseline_e20(2)/confusion_matrix_normalized.png (deflated 17%)
  adding: runs/detect/yolov8n_baseline_e20(2)/val_batch0_labels.jpg (deflated 9%)
  adding: runs/detect/yolov8n_baseline_e20(2)/train_batch1.jpg (deflated 8%)
  adding: runs/detect/yolov8n_baseline_e20(2)/BoxF1_curve.png (deflated 8%)
  adding: runs/detect/yolov8n_baseline_e20(2)/val_batch2_pred.jpg (deflated 9%)
  adding: runs/detect/yolov8n_baseline_e20(2)/BoxP_curve.png (deflated 8%)
  adding: runs/det

# 2차 학습

In [ ]:
# # 데이터 단일 탐색 코드
# target_file = 'K-003483-019861-025367-029667_0_2_0_2_90_000_200'

# print(f"--- 분석 대상: {target_file} ---")

# # 해당 파일 이름에 연결된 모든 어노테이션의 '원본 데이터'를 테이블로 출력
# try:
#     # 파일 이름으로 해당 이미지의 id들을 가져옴
#     img_infos = images_df[images_df['file_name'] == target_file]
#     img_ids = img_infos['id'].tolist()

#     # 해당 id들에 연결된 모든 어노테이션을 찾음
#     annotations_for_file = annotations_df[annotations_df['image_id'].isin(img_ids)]

#     if not annotations_for_file.empty:
#         print("\n[이 파일에 대한 전체 어노테이션 원본 데이터]")
#         display(annotations_for_file)

#         # 해당 파일만 시각화
#         print("\n[실제 시각화 결과]")
#         visualize_sample(images_df, annotations_df, categories_df, sample_files=[target_file])
#     else:
#         print("해당 파일 이름에 연결된 어노테이션을 찾을 수 없음")

# except Exception as e:
#     print(f"오류 발생: {e}")

In [ ]:
# calculate_iou - 바운딩 박스 계산 함수
def calculate_iou(box1, box2):
    x1_1, y1_1, w1, h1 = box1; x2_1, y2_1 = x1_1 + w1, y1_1 + h1
    x1_2, y1_2, w2, h2 = box2; x2_2, y2_2 = x1_2 + w2, y1_2 + h2
    xi_1, yi_1 = max(x1_1, x1_2), max(y1_1, y1_2)
    xi_2, yi_2 = min(x2_1, x2_2), min(y2_1, y2_2)
    inter_w, inter_h = max(0, xi_2 - xi_1), max(0, yi_2 - yi_1)
    inter_area = inter_w * inter_h
    area1, area2 = w1 * h1, w2 * h2
    union_area = area1 + area2 - inter_area
    return inter_area / union_area if union_area > 0 else 0

# -----------------------------------------------------

IOU_THRESHOLD_MIN = 0.2
all_overlaps = []

unique_files = images_df['file_name'].unique()
print(f"총 {len(unique_files)}개의 파일에 대해 IoU > {IOU_THRESHOLD_MIN}인 모든 겹침 사례탐색 ---#")

for file_name in tqdm(unique_files):
    img_infos = images_df[images_df['file_name'] == file_name]
    img_ids = img_infos['id'].tolist()
    anns_in_image = annotations_df[annotations_df['image_id'].isin(img_ids)]

    if len(anns_in_image) > 1:
        for ann1, ann2 in combinations(anns_in_image.iterrows(), 2):
            _, data1 = ann1
            _, data2 = ann2
            iou = calculate_iou(data1['bbox'], data2['bbox'])

            if iou > IOU_THRESHOLD_MIN:
                all_overlaps.append({
                    'file_name': file_name,
                    'ann_id_1': data1['id'], 'cat_id_1': data1['category_id'],
                    'ann_id_2': data2['id'], 'cat_id_2': data2['category_id'],
                    'iou': iou
                })

print("\n--- 탐지 및 분류 완료 ---")

if not all_overlaps:
    print("✅ IoU > 0.3인 겹침 사례를 찾지 못했습니다.")
else:
    overlaps_df = pd.DataFrame(all_overlaps)

    # IoU 값에 따라 두 그룹으로 분류
    perfect_duplicates_df = overlaps_df[overlaps_df['iou'] >= 0.95]
    partial_overlaps_df = overlaps_df[overlaps_df['iou'] < 0.95]

    # --- [개선된 부분] ---
    print(f"\n❗️ {len(perfect_duplicates_df)}개의 '완전 중복' 의심 사례를 발견했습니다.")
    if not perfect_duplicates_df.empty:
        display(perfect_duplicates_df)

    print(f"\n❗️ {len(partial_overlaps_df)}개의 '일부 겹침' 의심 사례를 발견했습니다.")
    if not partial_overlaps_df.empty:
        display(partial_overlaps_df)

총 1489개의 파일에 대해 IoU > 0.2인 모든 겹침 사례탐색 ---#


100%|██████████| 1489/1489 [00:03<00:00, 418.26it/s]


--- 탐지 및 분류 완료 ---

❗️ 5개의 '완전 중복' 의심 사례를 발견했습니다.


file_name  ann_id_1  cat_id_1  \
0      K-003351-003832-029667_0_2_0_2_90_000_200.png      2862      3831   
1      K-003351-020238-031863_0_2_0_2_70_000_200.png      3441     20237   
2      K-003351-018147-020238_0_2_0_2_90_000_200.png       916     20237   
4      K-003351-029667-031863_0_2_0_2_70_000_200.png       791      3350   
6  K-001900-016548-019607-033009_0_2_0_2_70_000_2...      4691     33008   

   ann_id_2  cat_id_2  iou  
0      2870     29666  1.0  
1      3444      3350  1.0  
2       910      3350  1.0  
4       788     31862  1.0  
6      4683     16547  1.0


❗️ 3개의 '일부 겹침' 의심 사례를 발견했습니다.


file_name  ann_id_1  cat_id_1  \
3  K-003483-027733-030308-036637_0_2_0_2_90_000_2...      3488     30307   
5  K-002483-003743-012081-019552_0_2_0_2_90_000_2...      2709      3742   
7  K-003483-019861-025367-029667_0_2_0_2_90_000_2...      3054     25366   

   ann_id_2  cat_id_2       iou  
3      3490     27732  0.340326  
5      2719     12080  0.492922  
7      3059     29666  0.292965

In [ ]:
# 바운딩 박스 중복/겹침 사례 시각화 함수
def visualize_overlaps(overlap_df, title_prefix, style='partial'):
    if overlap_df.empty:
        print(f"'{title_prefix}' 중복 사례 없음")
        return

    for index, row in overlap_df.iterrows():
        # 정보 추출
        file_name, iou = row['file_name'], row['iou']
        ann1 = annotations_df[annotations_df['id'] == row['ann_id_1']].iloc[0]
        ann2 = annotations_df[annotations_df['id'] == row['ann_id_2']].iloc[0]
        bbox1, bbox2 = ann1['bbox'], ann2['bbox']
        cat_name_1 = categories_df[categories_df['id'] == ann1['category_id']].iloc[0]['name']
        cat_name_2 = categories_df[categories_df['id'] == ann2['category_id']].iloc[0]['name']

        # 이미지 및 그래프 설정
        img_path = os.path.join('./data/train_images', file_name)
        img = Image.open(img_path).convert('RGB')
        fig, ax = plt.subplots(1, figsize=(8, 8))
        ax.imshow(img)
        ax.axis('off')
        title_font = fm.FontProperties(fname=font_path, size=12)
        ax.set_title(f"{title_prefix} (IoU: {iou:.3f})\n{file_name}", fontproperties=title_font)
        label_font = fm.FontProperties(fname=font_path, size=10)

        if style == 'duplicate':
            # 완전 중복: 파란 박스 1개 + 색상 라벨 2개
            rect = patches.Rectangle((bbox1[0], bbox1[1]), bbox1[2], bbox1[3], linewidth=2, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)
            ax.text(bbox1[0], bbox1[1] - 2, cat_name_1, color='white', backgroundcolor='blue', fontproperties=label_font, va='top')
            ax.text(bbox1[0], bbox1[1] + 15, cat_name_2, color='white', backgroundcolor='red', fontproperties=label_font, va='top')
        else: # partial
            # 일부 겹침: 청록색 박스 + 자홍색 박스
            rect1 = patches.Rectangle((bbox1[0], bbox1[1]), bbox1[2], bbox1[3], linewidth=2, edgecolor='cyan', facecolor='none')
            ax.add_patch(rect1)
            ax.text(bbox1[0], bbox1[1] - 2, cat_name_1, color='white', backgroundcolor='cyan', fontproperties=label_font, va='top')

            rect2 = patches.Rectangle((bbox2[0], bbox2[1]), bbox2[2], bbox2[3], linewidth=2, edgecolor='magenta', facecolor='none')
            ax.add_patch(rect2)
            ax.text(bbox2[0], bbox2[1] + 15, cat_name_2, color='white', backgroundcolor='magenta', fontproperties=label_font, va='top')

        plt.show()

# 실행
if 'perfect_duplicates_df' in locals():
    print("\n--- [사례 1] 완전 중복 (IoU >= 0.95) 시각화 ---")
    visualize_overlaps(perfect_duplicates_df, "완전 중복", style='duplicate')

if 'partial_overlaps_df' in locals():
    print("\n--- [사례 2] 일부 겹침 (0.2 < IoU < 0.95) 시각화 ---")
    visualize_overlaps(partial_overlaps_df, "일부 겹침", style='partial')

In [ ]:
# 'perfect_duplicates_df' -> IoU >= 0.95인 5개의 오류 목록 테이블
if 'perfect_duplicates_df' in locals() and not perfect_duplicates_df.empty:

    # 제거할 모든 어노테이션 ID들을 수집합니다.
    ids_to_remove_1 = perfect_duplicates_df['ann_id_1'].tolist()
    ids_to_remove_2 = perfect_duplicates_df['ann_id_2'].tolist()
    all_ids_to_remove = set(ids_to_remove_1 + ids_to_remove_2)

    print(f"제거 대상 어노테이션 ID 개수: {len(all_ids_to_remove)}")
    print(f"제거 전 annotations_df 크기: {len(annotations_df)}")

    # 원본 annotations_df에서 해당 ID들을 가진 행을 삭제합니다.
    annotations_df = annotations_df[~annotations_df['id'].isin(all_ids_to_remove)]

    print(f"제거 후 annotations_df 크기: {len(annotations_df)}")
    print("\n✅ 완전 중복 바운딩 박스 케이스가 제거되었습니다..")
else:
    print("error")

제거 대상 어노테이션 ID 개수: 10
제거 전 annotations_df 크기: 4526
제거 후 annotations_df 크기: 4516

✅ 완전 중복 바운딩 박스 케이스가 제거되었습니다..


In [ ]:
# @title YOLOv8s에 데이터 증강을 적용하여 훈련
model = YOLO('yolov8s.pt')

# 모델 훈련 시작
results = model.train(
    data='./data/dataset.yaml',
    epochs=20,
    imgsz=640,
    project='runs/detect',
    name='yolov8s_with_augmentation',
    exist_ok=True,

    # --- [데이터 증강 활성화] ---
    augment=True
)

print("\n✅ 모델 이름 : yolov8s + 데이터 증강 적용 모델 훈련")
print("이 모델의 훈련 결과는 'runs/detect/yolov8s_with_augmentation' 폴더에 저장했습니다.")


100%|██████████| 21.5M/21.5M [00:00<00:00, 272MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_with_augmentation, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

100%|██████████| 22.2M/22.2M [00:00<00:00, 43.5MB/s]

Overriding model.yaml nc=80 with nc=73

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

 22        [15, 18, 21]  1   2144299  ultralytics.nn.modules.head.Detect           [73, [128, 256, 512]]         
Model summary: 129 layers, 11,163,851 parameters, 11,163,835 gradients, 28.8 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all        296        884      0.767      0.978      0.859       0.85

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      4.62G     0.2087     0.5062     0.8027         22        640: 100%|██████████| 75/75 [00:46<00:00,  1.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.26it/s]


                   all        296        884      0.768      0.977      0.861      0.853

20 epochs completed in 0.309 hours.
Optimizer stripped from runs/detect/yolov8s_with_augmentation/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_with_augmentation/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_with_augmentation/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,153,835 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:10<00:00,  1.08s/it]


                   all        296        884      0.763      0.978      0.855       0.84
            보령부스파정 5mg         34         34      0.769          1      0.783      0.764
           뮤테란캡슐 100mg         36         36      0.796          1      0.835        0.8
           일양하이트린정 2mg         55         55      0.807          1      0.858      0.856
    기넥신에프정(은행엽엑스)(수출용)         93         93      0.808          1      0.847      0.843
     무코스타정(레바미피드)(비매품)         22         22      0.905          1      0.975      0.975
                  알드린정         11         11      0.676          1      0.739      0.725
          뉴로메드정(옥시라세탐)          4          4      0.706          1      0.995      0.921
            타이레놀정500mg          9          9       0.68      0.945      0.792      0.758
          에어탈정(아세클로페낙)          5          5      0.553          1      0.674      0.632
         삼남건조수산화알루미늄겔정         11         11      0.769          1      0.782       0.75
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# @title 모델의 증강 수치를 조금 더 세세하게 만지기
model = YOLO('yolov8s.pt')

results = model.train(
    data='./data/dataset.yaml',
    epochs=20,
    imgsz=640,
    project='runs/detect',
    name='yolov8s_custom_augmen',
    exist_ok=True,

    # 데이터증강 맞춤 추가
    augment=True,       # 증강 기능 자체는 켜두고,
    degrees=15,         # 회전 각도를 15도로 제한
    translate=0.1,      # 이동 범위를 10%로 제한
    scale=0.2,          # 크기 조절을 20%로 제한
    fliplr=0.5,         # 좌우 반전은 유용하므로 유지
    hsv_h=0.015,        # 색상(Hue) 변화는 최소화
    hsv_s=0.7,          # 채도(Saturation) 변화는 강하게 적용
    hsv_v=0.4           # 명도(Value)/밝기 변화는 강하게 적용
)

print("\n✅ 모델 이름 : yolov8s + 데이터 증강(맞춤) 적용 모델 훈련")
print("이 모델의 훈련 결과는 'runs/detect/yolov8s_custom_augmen' 폴더에 저장했습니다.")

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_custom_augmen, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning /content/data/train/labels.cache... 1191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1516.0±1298.0 MB/s, size: 1736.0 KB)


val: Scanning /content/data/val/labels.cache... 298 images, 0 backgrounds, 2 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]

...

                   all        296        884      0.773       0.98      0.858      0.831

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      4.79G     0.3135     0.5244     0.8184         22        640: 100%|██████████| 75/75 [00:46<00:00,  1.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:06<00:00,  1.43it/s]


                   all        296        884      0.767      0.976      0.859      0.833

20 epochs completed in 0.309 hours.
Optimizer stripped from runs/detect/yolov8s_custom_augmen/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_custom_augmen/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_custom_augmen/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,153,835 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


                   all        296        884      0.747      0.973      0.871      0.832
            보령부스파정 5mg         34         34      0.739          1      0.837      0.772
           뮤테란캡슐 100mg         36         36      0.797          1      0.885      0.837
           일양하이트린정 2mg         55         55      0.737          1      0.878      0.793
    기넥신에프정(은행엽엑스)(수출용)         93         93      0.807          1      0.829      0.802
     무코스타정(레바미피드)(비매품)         22         22      0.906          1      0.932      0.932
                  알드린정         11         11      0.678          1      0.838      0.817
          뉴로메드정(옥시라세탐)          4          4      0.641          1      0.995      0.952
            타이레놀정500mg          9          9      0.688          1      0.886       0.84
          에어탈정(아세클로페낙)          5          5      0.541          1      0.818      0.764
         삼남건조수산화알루미늄겔정         11         11      0.781          1      0.826      0.765
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# @title 모델에 증강없이 조기종료
model = YOLO('yolov8s.pt')

results = model.train(
    data='./data/dataset.yaml',
    epochs=100,
    imgsz=640,
    project='runs/detect',
    name='yolov8s_final_e100', # 최종 모델 훈련
    exist_ok=True,
    augment=False,
    patience=25
)

print("\n✅ 최종 모델 훈련이 완료되었습니다.")

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_final_e100, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0.0, plots=True, pose=12.0, pretrai

train: Scanning /content/data/train/labels.cache... 1191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1710.5±1739.5 MB/s, size: 1736.0 KB)


val: Scanning /content/data/val/labels.cache... 298 images, 0 backgrounds, 2 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     50/100      4.84G     0.2422      0.475     0.8293         35        640: 100%|██████████| 75/75 [00:47<00:00,  1.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.31it/s]


                   all        296        884      0.761      0.967      0.873      0.865
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 25, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

50 epochs completed in 0.789 hours.
Optimizer stripped from runs/detect/yolov8s_final_e100/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_final_e100/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_final_e100/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,153,835 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.06it/s]


                   all        296        884      0.766      0.982      0.878      0.867
            보령부스파정 5mg         34         34      0.765          1      0.816      0.814
           뮤테란캡슐 100mg         36         36      0.793          1      0.855      0.832
           일양하이트린정 2mg         55         55      0.802          1      0.892      0.891
    기넥신에프정(은행엽엑스)(수출용)         93         93      0.805          1      0.853      0.847
     무코스타정(레바미피드)(비매품)         22         22      0.896          1      0.946      0.946
                  알드린정         11         11      0.669          1      0.976      0.943
          뉴로메드정(옥시라세탐)          4          4      0.679          1      0.995      0.958
            타이레놀정500mg          9          9      0.667          1      0.794      0.794
          에어탈정(아세클로페낙)          5          5      0.527          1      0.938      0.938
         삼남건조수산화알루미늄겔정         11         11      0.757          1       0.85      0.798
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# @title 모델에 증강0, 조기종료
model = YOLO('yolov8s.pt')

results = model.train(
    data='./data/dataset.yaml',
    epochs=100,
    imgsz=640,
    patience=25,
    project='runs/detect',
    name='yolov8s_custom_augment_e100',
    exist_ok=True,

    # --- [데이터 증강 옵션] ---
    augment=True,
    degrees=15,         # 회전 각도를 15도로 제한
    translate=0.1,      # 이동 범위를 10%로 제한
    scale=0.2,          # 크기 조절을 20%로 제한
    fliplr=0.5,         # 좌우 반전은 유지
    hsv_h=0.015,        # 색상(Hue) 변화는 최소화
    hsv_s=0.7,          # 채도(Saturation) 변화는 강하게 적용
    hsv_v=0.4           # 명도(Value)/밝기 변화는 강하게 적용
)

print("\n✅ 맞춤형 데이터 증강 훈련이 완료되었습니다.")
print("훈련 결과는 'runs/detect/yolov8s_custom_augment_e100' 폴더에 저장되었습니다.")


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/dataset.yaml, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_custom_augment_e100, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=25, perspective=0.0, plots=True, pose=12.0, 

train: Scanning /content/data/train/labels.cache... 1191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1191/1191 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1035.7±1092.7 MB/s, size: 1736.0 KB)


val: Scanning /content/data/val/labels.cache... 298 images, 0 backgrounds, 2 corrupt: 100%|██████████| 298/298 [00:00<?, ?it/s]

...

                   all        296        884      0.756      0.978      0.854      0.842

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     52/100       4.8G     0.3279     0.5395     0.8629         36        640: 100%|██████████| 75/75 [00:47<00:00,  1.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:08<00:00,  1.22it/s]


                   all        296        884      0.748      0.987      0.852       0.84
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 27, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

52 epochs completed in 0.833 hours.
Optimizer stripped from runs/detect/yolov8s_custom_augment_e100/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_custom_augment_e100/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_custom_augment_e100/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,153,835 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


                   all        296        884      0.739      0.969      0.866      0.822
            보령부스파정 5mg         34         34      0.763          1      0.775      0.707
           뮤테란캡슐 100mg         36         36      0.794          1       0.88      0.775
           일양하이트린정 2mg         55         55      0.792          1      0.845      0.801
    기넥신에프정(은행엽엑스)(수출용)         93         93      0.805          1      0.814      0.776
     무코스타정(레바미피드)(비매품)         22         22      0.902          1      0.973      0.945
                  알드린정         11         11      0.783      0.658      0.836      0.825
          뉴로메드정(옥시라세탐)          4          4      0.569          1      0.995      0.958
            타이레놀정500mg          9          9          1          0      0.915      0.874
          에어탈정(아세클로페낙)          5          5      0.525          1      0.735      0.711
         삼남건조수산화알루미늄겔정         11         11      0.768          1      0.802      0.765
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# 모델 결과 저장
!zip -r runs.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/yolov8s_custom_augmen/ (stored 0%)
  adding: runs/detect/yolov8s_custom_augmen/val_batch2_pred.jpg (deflated 8%)
  adding: runs/detect/yolov8s_custom_augmen/train_batch751.jpg (deflated 12%)
  adding: runs/detect/yolov8s_custom_augmen/args.yaml (deflated 53%)
  adding: runs/detect/yolov8s_custom_augmen/val_batch2_labels.jpg (deflated 8%)
  adding: runs/detect/yolov8s_custom_augmen/weights/ (stored 0%)
  adding: runs/detect/yolov8s_custom_augmen/weights/last.pt (deflated 8%)
  adding: runs/detect/yolov8s_custom_augmen/weights/best.pt (deflated 8%)
  adding: runs/detect/yolov8s_custom_augmen/confusion_matrix.png (deflated 17%)
  adding: runs/detect/yolov8s_custom_augmen/BoxPR_curve.png (deflated 14%)
  adding: runs/detect/yolov8s_custom_augmen/train_batch0.jpg (deflated 6%)
  adding: runs/detect/yolov8s_custom_augmen/train_batch750.jpg (deflated 12%)
  adding: runs/detect/yolov8s_custom_augmen/train_batc

In [ ]:
# @title 평가
model_path = '/content/best.pt'
model = YOLO(model_path)
print(f"✅ '{model_path}'을 평가.")

reverse_category_id_map = {v: k for k, v in category_id_map.items()}
print("✅ 클래스 ID 복원을 위한 맵 생성")

test_images_dir = './data/test_images'
print(f"\n'{test_images_dir}' 폴더의 이미지에 대한 예측을 시작합니다...")
results = model.predict(source=test_images_dir, stream=True)
print("✅ 예측 완료")

submission_data = []
annotation_id_counter = 1

for result in tqdm(results):
    file_name = os.path.basename(result.path)
    image_id_match = re.search(r'\d+', file_name)
    if not image_id_match: continue
    image_id = int(image_id_match.group())

    for box in result.boxes:
        x_center, y_center, width, height = box.xywh[0].tolist()
        x_min, y_min = x_center - width / 2, y_center - height / 2

        yolo_class_id = int(box.cls[0])
        original_category_id = reverse_category_id_map.get(yolo_class_id, -1) # 혹시 모를 오류 방지

        score = float(box.conf[0])

        submission_data.append([
            annotation_id_counter,
            image_id,
            original_category_id, # 복원된 원본 ID를 사용
            round(x_min, 3), round(y_min, 3),
            round(width, 3), round(height, 3),
            round(score, 3)
        ])
        annotation_id_counter += 1

submission_df = pd.DataFrame(submission_data,
                             columns=['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score'])

submission_df.to_csv('submission_corrected.csv', index=False)

print("\n✅ 'submission_corrected.csv' 파일 생성")
display(submission_df.head())

✅ '/content/best.pt'을 평가.
✅ 클래스 ID 복원을 위한 맵 생성

'./data/test_images' 폴더의 이미지에 대한 예측을 시작합니다...
✅ 예측 완료


0it [00:00, ?it/s]


image 1/843 /content/data/test_images/1.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 놀텍정 10mg, 1 울트라셋이알서방정, 18.7ms


1it [00:00,  4.06it/s]

image 2/843 /content/data/test_images/10.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 12.5ms


...

842it [00:55, 16.17it/s]

image 843/843 /content/data/test_images/999.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 라비에트정 20mg, 1 레일라정, 12.4ms
Speed: 2.5ms preprocess, 12.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 512)


843it [00:55, 15.21it/s]



✅ 'submission_corrected.csv' 파일 생성


annotation_id  image_id  category_id   bbox_x   bbox_y   bbox_w   bbox_h  \
0              1         1        27925  599.887  674.818  255.479  478.955   
1              2         1        16550  555.611   68.888  398.114  405.786   
2              3         1         1899  160.413  251.698  200.497  125.070   
3              4         1        24849  173.560  741.580  179.317  289.353   
4              5        10        21770  641.426  289.936  190.554  182.407   

   score  
0  0.875  
1  0.869  
2  0.801  
3  0.427  
4  0.958

In [ ]:
# 2차 데이터 전처리 후 증강 없는 모델 평가
model_path = '/content/best.pt'
model = YOLO(model_path)
print(f"✅ '{model_path}'을 평가.")

reverse_category_id_map = {v: k for k, v in category_id_map.items()}
print("✅ 클래스 ID 복원을 위한 맵 생성")


test_images_dir = './data/test_images'
print(f"\n'{test_images_dir}' 폴더의 이미지에 대한 예측을 시작합니다...")
results = model.predict(source=test_images_dir, stream=True)
print("✅ 예측 완료")

submission_data = []
annotation_id_counter = 1

for result in tqdm(results):
    file_name = os.path.basename(result.path)
    image_id_match = re.search(r'\d+', file_name)
    if not image_id_match: continue
    image_id = int(image_id_match.group())

    for box in result.boxes:
        x_center, y_center, width, height = box.xywh[0].tolist()
        x_min, y_min = x_center - width / 2, y_center - height / 2

        # 모델이 예측한 yolo_id를 원본 category_id로 복원
        yolo_class_id = int(box.cls[0])
        original_category_id = reverse_category_id_map.get(yolo_class_id, -1) # 혹시 모를 오류 방지

        score = float(box.conf[0])

        submission_data.append([
            annotation_id_counter,
            image_id,
            original_category_id, # 복원된 원본 ID를 사용
            round(x_min, 3), round(y_min, 3),
            round(width, 3), round(height, 3),
            round(score, 3)
        ])
        annotation_id_counter += 1

submission_df = pd.DataFrame(submission_data,
                             columns=['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score'])

submission_df.to_csv('submission_corrected.csv', index=False)

print("\n✅ 'submission_corrected.csv' 파일 생성")
display(submission_df.head())

✅ '/content/best.pt'을 평가.
✅ 클래스 ID 복원을 위한 맵 생성

'./data/test_images' 폴더의 이미지에 대한 예측을 시작합니다...
✅ 예측 완료


0it [00:00, ?it/s]


image 1/843 /content/data/test_images/1.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 놀텍정 10mg, 1 울트라셋이알서방정, 13.0ms
image 2/843 /content/data/test_images/10.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 12.3ms


2it [00:00, 15.19it/s]

image 3/843 /content/data/test_images/100.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 란스톤엘에프디티정 30mg, 1 신바로정, 12.3ms
image 4/843 /content/data/test_images/1003.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 12.3ms


...

843it [00:48, 17.36it/s]

Speed: 2.4ms preprocess, 12.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 512)


843it [00:48, 17.47it/s]


✅ 'submission_corrected.csv' 파일 생성


annotation_id  image_id  category_id   bbox_x   bbox_y   bbox_w   bbox_h  \
0              1         1        27925  600.880  674.418  250.025  479.874   
1              2         1        16550  557.367   70.626  395.857  404.716   
2              3         1         1899  160.200  251.509  198.975  125.478   
3              4         1        24849  173.572  739.241  180.144  290.979   
4              5        10        21770  643.250  288.670  188.426  186.553   

   score  
0  0.976  
1  0.771  
2  0.759  
3  0.573  
4  0.911

# 3차 - 영호님 데이터로 학습

In [ ]:
# 새로운 데이터의 경로 설정
annotations_path = '/content/pill_data_v3/raw_data/train_annotations'
images_path = '/content/pill_data_v3/raw_data/train_images'

# 데이터를 담을 리스트 초기화
images_list = []
annotations_list = []
categories_list = []

# 중복을 피하기 위한 ID 추적용 집합(set) 및 딕셔너리
seen_image_ids = set()
seen_category_ids = {}

print(f"'{annotations_path}' 폴더의 모든 .json 파일을 읽어옵니다.")

# tqdm을 사용하여 진행 상황 표시
for root, _, files in tqdm(os.walk(annotations_path), desc="Processing folders"):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                data = json.load(f)

                # 이미지 정보 추가 (중복 방지)
                if 'images' in data:
                    for img in data['images']:
                        if img['id'] not in seen_image_ids:
                            images_list.append(img)
                            seen_image_ids.add(img['id'])

                # 카테고리 정보 추가 (중복 방지)
                if 'categories' in data:
                    for cat in data['categories']:
                        if cat['id'] not in seen_category_ids:
                            categories_list.append(cat)
                            seen_category_ids[cat['id']] = cat['name']

                # 어노테이션 정보 추가 (어노테이션은 항상 고유함)
                if 'annotations' in data:
                    annotations_list.extend(data['annotations'])

print("\n데이터 통합 완료. Pandas DataFrame으로 변환합니다.")

# 리스트를 DataFrame으로 변환
images_df = pd.DataFrame(images_list)
annotations_df = pd.DataFrame(annotations_list)
categories_df = pd.DataFrame(categories_list)

print("\n✅ 새로운 데이터셋에 대한 DataFrame 생성이 완료되었습니다.")
print("\n--- 생성된 DataFrame 정보 ---")
print(f"Images DataFrame: {images_df.shape[0]}개 이미지 정보")
print(f"Annotations DataFrame: {annotations_df.shape[0]}개 어노테이션")
print(f"Categories DataFrame: {categories_df.shape[0]}개 클래스")

'/content/pill_data_v3/raw_data/train_annotations' 폴더의 모든 .json 파일을 읽어옵니다.


Processing folders: 2387it [00:01, 1591.10it/s]



데이터 통합 완료. Pandas DataFrame으로 변환합니다.

✅ 새로운 데이터셋에 대한 DataFrame 생성이 완료되었습니다.

--- 생성된 DataFrame 정보 ---
Images DataFrame: 1486개 이미지 정보
Annotations DataFrame: 4547개 어노테이션
Categories DataFrame: 74개 클래스


In [ ]:
from sklearn.model_selection import train_test_split

# 새로운 images_df에서 고유 이미지 파일 목록을 가져옴
unique_images_v3 = images_df['file_name'].unique()
print(f"새로운 데이터셋의 고유 이미지 수: {len(unique_images_v3)}")

# 이미지 목록을 훈련용과 검증용으로 8:2 비율로 나눔
train_files_v3, val_files_v3 = train_test_split(unique_images_v3, test_size=0.2, random_state=42)

print(f"훈련용 이미지 수: {len(train_files_v3)}")
print(f"검증용 이미지 수: {len(val_files_v3)}")

새로운 데이터셋의 고유 이미지 수: 1485
훈련용 이미지 수: 1188
검증용 이미지 수: 297


In [ ]:
base_dir_v3 = '/content/data_v3'
os.makedirs(os.path.join(base_dir_v3, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(base_dir_v3, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(base_dir_v3, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(base_dir_v3, 'val/labels'), exist_ok=True)

category_names_v3 = sorted(categories_df['name'].unique())
category_id_map_v3 = {cat_id: i for i, cat_id in enumerate(categories_df.set_index('name').loc[category_names_v3]['id'])}
print("✅ 새로운 클래스 ID 맵(74개)이 생성되었습니다.")


def convert_to_yolo_format(files, source_img_path, target_img_path, target_label_path):
    for file_name in tqdm(files, desc=f"Processing {os.path.basename(target_img_path)}"):
        # 이미지 정보 찾기
        img_info_rows = images_df[images_df['file_name'] == file_name]
        if img_info_rows.empty:
            continue

        # 동일 파일 이름에 여러 ID가 있을 수 있으므로 첫 번째 정보를 기준으로 사용
        img_info = img_info_rows.iloc[0]
        img_id = img_info['id']
        img_w, img_h = img_info['width'], img_info['height']

        # 해당 이미지에 대한 모든 어노테이션 찾기
        ann_ids = images_df[images_df['file_name'] == file_name]['id'].tolist()
        annotations = annotations_df[annotations_df['image_id'].isin(ann_ids)]

        if annotations.empty:
            continue

        # 라벨 파일 생성
        label_file_path = os.path.join(target_label_path, os.path.splitext(file_name)[0] + '.txt')
        with open(label_file_path, 'w') as f:
            for _, ann in annotations.iterrows():
                cat_id = ann['category_id']
                if cat_id in category_id_map_v3:
                    yolo_class_id = category_id_map_v3[cat_id]

                    # YOLO 좌표 계산 (x_center, y_center, width, height)
                    x, y, w, h = ann['bbox']
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    norm_w = w / img_w
                    norm_h = h / img_h

                    # 좌표값이 0~1 범위를 벗어나지 않도록 클리핑
                    x_center = max(0, min(1, x_center))
                    y_center = max(0, min(1, y_center))
                    norm_w = max(0, min(1, norm_w))
                    norm_h = max(0, min(1, norm_h))

                    f.write(f"{yolo_class_id} {x_center} {y_center} {norm_w} {norm_h}\n")

        # 이미지 파일 복사
        shutil.copy(os.path.join(images_path, file_name), os.path.join(target_img_path, file_name))

convert_to_yolo_format(train_files_v3, images_path, os.path.join(base_dir_v3, 'train/images'), os.path.join(base_dir_v3, 'train/labels'))
convert_to_yolo_format(val_files_v3, images_path, os.path.join(base_dir_v3, 'val/images'), os.path.join(base_dir_v3, 'val/labels'))

class_names_ordered = [name for name, _ in sorted(category_id_map_v3.items(), key=lambda item: item[1])]

yaml_data_v3 = {
    'train': os.path.abspath(os.path.join(base_dir_v3, 'train/images')),
    'val': os.path.abspath(os.path.join(base_dir_v3, 'val/images')),
    'nc': len(class_names_ordered),
    'names': class_names_ordered
}

yaml_path_v3 = os.path.join(base_dir_v3, 'dataset_v3.yaml')
with open(yaml_path_v3, 'w') as f:
    yaml.dump(yaml_data_v3, f, default_flow_style=False, allow_unicode=True)

print(f"YAML 파일 위치: {yaml_path_v3}")

✅ 새로운 클래스 ID 맵(74개)이 생성되었습니다.


Processing images: 100%|██████████| 297/297 [00:05<00:00, 52.76it/s]



✅ 새로운 데이터셋(v3)에 대한 YOLO 포맷 변환 및 환경 구성이 완료되었습니다.
YAML 파일 위치: /content/data_v3/dataset_v3.yaml


In [ ]:

sorted_categories_df_v3 = categories_df.sort_values(by='id').reset_index(drop=True)


category_id_map_v3 = {row['id']: index for index, row in sorted_categories_df_v3.iterrows()}

# --- 2. 새로운 데이터셋을 저장할 폴더 재생성 ---
base_dir_v3 = '/content/data_v3'
if os.path.exists(base_dir_v3):
    shutil.rmtree(base_dir_v3) # 기존 폴더 삭제
os.makedirs(os.path.join(base_dir_v3, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(base_dir_v3, 'train/labels'), exist_ok=True)
os.makedirs(os.path.join(base_dir_v3, 'val/images'), exist_ok=True)
os.makedirs(os.path.join(base_dir_v3, 'val/labels'), exist_ok=True)

# --- 3. 수정된 ID 맵으로 YOLO 라벨 파일 다시 생성 ---
convert_to_yolo_format(train_files_v3, images_path, os.path.join(base_dir_v3, 'train/images'), os.path.join(base_dir_v3, 'train/labels'))
convert_to_yolo_format(val_files_v3, images_path, os.path.join(base_dir_v3, 'val/images'), os.path.join(base_dir_v3, 'val/labels'))
print("✅ 라벨 파일 재생성이 완료되었습니다.")

# --- 4. 수정된 ID 맵으로 dataset_v3.yaml 파일 다시 생성 ---
class_names_ordered_v3 = sorted_categories_df_v3['name'].tolist()

yaml_data_v3 = {
    'train': os.path.abspath(os.path.join(base_dir_v3, 'train/images')),
    'val': os.path.abspath(os.path.join(base_dir_v3, 'val/images')),
    'nc': len(class_names_ordered_v3),
    'names': class_names_ordered_v3
}

yaml_path_v3 = os.path.join(base_dir_v3, 'dataset_v3.yaml')
with open(yaml_path_v3, 'w') as f:
    yaml.dump(yaml_data_v3, f, default_flow_style=False, allow_unicode=True)

print(f"✅ 새로운 'dataset_v3.yaml' 파일 생성이 완료되었습니다: {yaml_path_v3}")

✅ 안정적인 v3 데이터(74개 클래스) ID 맵 생성을 완료했습니다.

수정된 ID 맵을 사용하여 라벨 파일 및 이미지 복사를 다시 시작합니다...


Processing images: 100%|██████████| 297/297 [00:06<00:00, 46.58it/s]

✅ 라벨 파일 재생성이 완료되었습니다.
✅ 새로운 'dataset_v3.yaml' 파일 생성이 완료되었습니다: /content/data_v3/dataset_v3.yaml


In [ ]:
def visualize_sample(images_df, annotations_df, categories_df, num_samples=None, sample_files=None):
    """
    데이터셋에서 이미지를 시각화하는 함수 (이전에 만들었던 최종 버전)
    """
    files_to_show = []
    if num_samples is not None:
        unique_image_files = images_df['file_name'].unique()
        files_to_show = random.sample(list(unique_image_files), num_samples)
    elif sample_files is not None:
        files_to_show = sample_files
    else:
        print("에러: 파일이 제공되지 않았습니다. ")
        return

    for file_name in files_to_show:
        # 파일 이름으로 이미지 정보 찾기
        image_info_rows = images_df[images_df['file_name'] == file_name]
        if image_info_rows.empty:
            print(f"‼️ {file_name}에 대한 이미지 정보를 찾을 수 없습니다.")
            continue

        # 해당 이미지에 속한 모든 어노테이션 찾기
        img_ids = image_info_rows['id'].tolist()
        ann_infos = annotations_df[annotations_df['image_id'].isin(img_ids)]

        # 이미지 열기 및 그래프 설정
        img_path = os.path.join('/content/pill_data_v3/raw_data/train_images', file_name) # 새로운 이미지 경로
        if not os.path.exists(img_path):
            print(f"경고: 이미지 파일을 찾을 수 없습니다 - {img_path}")
            continue

        img = Image.open(img_path).convert('RGB')
        fig, ax = plt.subplots(1, figsize=(10, 10))
        ax.imshow(img)
        ax.axis('off')

        title_font = fm.FontProperties(fname=font_path, size=15)
        ax.set_title(f"Image: {file_name}", fontproperties=title_font)

        # 바운딩 박스 및 라벨 그리기
        for _, ann in ann_infos.iterrows():
            bbox = ann['bbox']
            cat_id = ann['category_id']
            cat_name = categories_df[categories_df['id'] == cat_id]['name'].values[0]

            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],
                                     linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)

            label_font = fm.FontProperties(fname=font_path, size=12)
            ax.text(bbox[0], bbox[1] - 10, cat_name,
                    color='white', backgroundcolor='lime',
                    fontproperties=label_font)
        plt.show()

In [ ]:
# 검증할 이미지 폴더 경로
val_img_dir_v3 = '/content/data_v3/val/images'
val_images_list_v3 = [f for f in os.listdir(val_img_dir_v3) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 무작위로 3개의 파일을 선택
if len(val_images_list_v3) > 3:
    random_samples = random.sample(val_images_list_v3, 3)
else:
    random_samples = val_images_list_v3

visualize_sample(images_df, annotations_df, categories_df, sample_files=random_samples)

In [ ]:
champion_model_path = '/content/best.pt'
model = YOLO(champion_model_path)

results = model.train(
    data='/content/data_v3/dataset_v3.yaml',
    epochs=150,
    patience=30,
    imgsz=640,
    project='runs/detect',
    name='yolov8s_finetuned_on_v3',
    exist_ok=True
)

print("\n✅ 새로운 데이터셋에 대한 Fine-tuning 훈련이 완료되었습니다.")
print(f"훈련 결과는 'runs/detect/yolov8s_finetuned_on_v3_new' 폴더에서 확인하실 수 있습니다.")

✅ 이전 챔피언 모델 '/content/best.pt'을(를) 성공적으로 불러왔습니다.
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data_v3/dataset_v3.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_finetuned_on_v3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

100%|██████████| 22.2M/22.2M [00:00<00:00, 79.3MB/s]

Overriding model.yaml nc=73 with nc=74

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

  8                  -1  1   1838080  ultralytics.nn.modules.block.C2f             [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    591360  ultralytics.nn.modules.block.C2f             [768, 256, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 16                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128,

train: Scanning /content/data_v3/train/labels... 1188 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1188/1188 [00:13<00:00, 87.33it/s]

...

     83/150      4.85G     0.2212     0.4377     0.8304         16        640: 100%|██████████| 75/75 [00:49<00:00,  1.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:07<00:00,  1.31it/s]

                   all        297        912      0.784      0.967      0.877      0.869
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 53, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



83 epochs completed in 1.357 hours.
Optimizer stripped from runs/detect/yolov8s_finetuned_on_v3/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_finetuned_on_v3/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_finetuned_on_v3/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,154,222 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:08<00:00,  1.12it/s]


                   all        297        912       0.79      0.966      0.896       0.89
            보령부스파정 5mg         38         38       0.82          1      0.873      0.867
           뮤테란캡슐 100mg         40         40      0.764          1      0.846      0.825
           일양하이트린정 2mg         48         48       0.77          1      0.802      0.802
    기넥신에프정(은행엽엑스)(수출용)         93         93      0.874          1      0.889      0.889
     무코스타정(레바미피드)(비매품)         25         25      0.828       0.96      0.892      0.892
                  알드린정         15         15      0.816          1      0.976      0.976
          뉴로메드정(옥시라세탐)          6          6      0.796          1      0.876      0.876
            타이레놀정500mg         11         11      0.896          1      0.942      0.942
          에어탈정(아세클로페낙)          7          7       0.68          1      0.852      0.852
         삼남건조수산화알루미늄겔정         13         13      0.705          1      0.876      0.876
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
yaml_path_v3 = '/content/data_v3/dataset_v3.yaml'
try:
    with open(yaml_path_v3, 'r', encoding='utf-8') as f:
        data_yaml = yaml.safe_load(f)
    yolo_class_names = data_yaml['names']
    print("✅ 훈련에 사용된 YAML 파일에서 클래스 이름 목록을 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"❗️ YAML 파일 로딩 오류: {e}")
    yolo_class_names = None

# 클래스 이름과 원본 ID를 매핑하는 딕셔너리 생성
if 'categories_df' in locals() and yolo_class_names is not None:
    name_to_original_id_map = pd.Series(categories_df.id.values, index=categories_df.name).to_dict()
    print("✅ 클래스 이름 -> 원본 ID 맵 생성이 완료되었습니다.")

    # 새로운 챔피언 모델 로드
    model_path = '/content/runs/detect/yolov8s_finetuned_on_v3/weights/best.pt'
    model = YOLO(model_path)
    print(f"✅ 새로운 챔피언 모델 '{model_path}'을(를) 성공적으로 불러왔습니다.")

    # Test Set 예측 실행
    test_images_dir = '/content/pill_data_v3/raw_data/test_images'
    print(f"\n'{test_images_dir}' 폴더의 이미지에 대한 예측을 시작합니다...")
    results = model.predict(source=test_images_dir, stream=True)
    print("✅ 예측이 완료되었습니다.")

    # 제출 파일 생성
    print("\n제출 파일 생성을 시작합니다...")
    submission_data = []
    annotation_id_counter = 1

    for result in tqdm(results, desc="파일 생성 중"):
        file_name = os.path.basename(result.path)
        image_id_match = re.search(r'\d+', file_name)
        if not image_id_match: continue
        image_id = int(image_id_match.group())

        for box in result.boxes:
            x_center, y_center, width, height = box.xywh[0].tolist()
            x_min, y_min = x_center - width / 2, y_center - height / 2

            # YAML 파일 기준의 정확한 ID 복원 로직
            yolo_class_id = int(box.cls[0])
            class_name = yolo_class_names[yolo_class_id]
            original_category_id = name_to_original_id_map.get(class_name, -1)

            score = float(box.conf[0])

            submission_data.append([
                annotation_id_counter, image_id, original_category_id,
                round(x_min, 3), round(y_min, 3), round(width, 3), round(height, 3),
                round(score, 3)
            ])
            annotation_id_counter += 1

    submission_df = pd.DataFrame(submission_data,
                                 columns=['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score'])

    submission_df.to_csv('submission_finetuned_v3.csv', index=False)

    print("\n✅ 'submission_finetuned_v3.csv' 파일 생성이 완료되었습니다.")
    display(submission_df.head())

else:
    print("❗️'categories_df'가 없습니다.")

✅ 훈련에 사용된 YAML 파일에서 클래스 이름 목록을 성공적으로 불러왔습니다.
✅ 클래스 이름 -> 원본 ID 맵 생성이 완료되었습니다.
✅ 새로운 챔피언 모델 '/content/runs/detect/yolov8s_finetuned_on_v3/weights/best.pt'을(를) 성공적으로 불러왔습니다.

'/content/pill_data_v3/raw_data/test_images' 폴더의 이미지에 대한 예측을 시작합니다...
✅ 예측이 완료되었습니다.

제출 파일 생성을 시작합니다...


파일 생성 중: 0it [00:00, ?it/s]


image 1/843 /content/pill_data_v3/raw_data/test_images/1.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 놀텍정 10mg, 1 울트라셋이알서방정, 13.2ms


파일 생성 중: 1it [00:00,  6.31it/s]

image 2/843 /content/pill_data_v3/raw_data/test_images/10.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 12.4ms
image 3/843 /content/pill_data_v3/raw_data/test_images/100.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 란스톤엘에프디티정 30mg, 1 신바로정, 12.3ms


...

파일 생성 중: 843it [00:58, 15.33it/s]

Speed: 2.6ms preprocess, 12.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 512)


파일 생성 중: 843it [00:58, 14.34it/s]


✅ 'submission_finetuned_v3.csv' 파일 생성이 완료되었습니다.


annotation_id  image_id  category_id   bbox_x   bbox_y   bbox_w   bbox_h  \
0              1         1        16550  554.376   74.192  404.490  398.850   
1              2         1        27925  602.393  671.605  250.736  482.257   
2              3         1        24849  170.999  739.122  182.650  292.890   
3              4         1         1899  157.983  251.071  203.099  125.547   
4              5        10        29450  103.944  236.805  409.273  221.000   

   score  
0  0.855  
1  0.852  
2  0.775  
3  0.722  
4  0.900

In [ ]:
# 2. 중강을 약하게 줘 본 모델 학습
champion_model_path = '/content/best.pt'
model = YOLO(champion_model_path)
print(f"✅ 이전 챔피언 모델 '{champion_model_path}'을(를) 성공적으로 불러왔습니다.")

results = model.train(
    data='/content/data_v3/dataset_v3.yaml',
    epochs=150,
    patience=30,
    imgsz=640,
    project='runs/detect',
    name='yolov8s_finetuned_on_v3_cuAG',
    exist_ok=True
)

print("\n✅ 새로운 데이터셋에 대한 훈련이 완료되었습니다.")
print(f"훈련 결과는 'runs/detect/yolov8s_finetuned_on_v3_cutomAG_new' 폴더에서 확인하실 수 있습니다.")

✅ 이전 챔피언 모델 '/content/best.pt'을(를) 성공적으로 불러왔습니다.
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data_v3/dataset_v3.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_finetuned_on_v3_cuAG, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

train: Scanning /content/data_v3/train/labels.cache... 1188 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1188/1188 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2554.4±1624.0 MB/s, size: 1774.4 KB)


val: Scanning /content/data_v3/val/labels.cache... 297 images, 0 backgrounds, 0 corrupt: 100%|██████████| 297/297 [00:00<?, ?it/s]


...

                   all        297        912       0.77      0.991      0.887      0.879

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     39/150      4.76G     0.2588     0.5012     0.8391         24        640: 100%|██████████| 75/75 [00:50<00:00,  1.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


                   all        297        912      0.781      0.978      0.888      0.878
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

39 epochs completed in 0.652 hours.
Optimizer stripped from runs/detect/yolov8s_finetuned_on_v3_cuAG/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/yolov8s_finetuned_on_v3_cuAG/weights/best.pt, 22.6MB

Validating runs/detect/yolov8s_finetuned_on_v3_cuAG/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,154,222 parameters, 0 gradients, 28.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:10<00:00,  1.04s/it]


                   all        297        912      0.756      0.987      0.898      0.884
            보령부스파정 5mg         38         38      0.813          1      0.907      0.872
           뮤테란캡슐 100mg         40         40      0.759          1      0.892      0.841
           일양하이트린정 2mg         48         48      0.764          1      0.803      0.803
    기넥신에프정(은행엽엑스)(수출용)         93         93      0.871          1      0.909      0.909
     무코스타정(레바미피드)(비매품)         25         25      0.787          1      0.848      0.848
                  알드린정         15         15      0.806          1      0.964      0.964
          뉴로메드정(옥시라세탐)          6          6      0.755          1      0.924      0.924
            타이레놀정500mg         11         11      0.863          1      0.995      0.984
          에어탈정(아세클로페낙)          7          7      0.652          1      0.852      0.852
         삼남건조수산화알루미늄겔정         13         13      0.706          1      0.863      0.863
타이레놀이알서방정(아세트아미노펜)(수출

In [ ]:
# 모델이 인식하는 0~73번 클래스의 가장 정확한 순서 정보
yaml_path_v3 = '/content/data_v3/dataset_v3.yaml'
try:
    with open(yaml_path_v3, 'r', encoding='utf-8') as f:
        data_yaml = yaml.safe_load(f)
    yolo_class_names = data_yaml['names']
    print("✅ 훈련에 사용된 YAML 파일에서 클래스 이름 목록을 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"❗️ YAML 파일 로딩 오류: {e}")
    yolo_class_names = None

# 클래스 이름과 원본 ID를 매핑하는 딕셔너리 생성
if 'categories_df' in locals() and yolo_class_names is not None:
    name_to_original_id_map = pd.Series(categories_df.id.values, index=categories_df.name).to_dict()
    print("✅ 클래스 이름 -> 원본 ID 맵 생성이 완료되었습니다.")

    # 새로운 챔피언 모델 로드
    model_path = '/content/runs/detect/yolov8s_finetuned_on_v3_cuAG/weights/best.pt'
    model = YOLO(model_path)
    print(f"✅ 새로운 챔피언 모델 '{model_path}'을(를) 성공적으로 불러왔습니다.")

    # Test Set 예측 실행
    test_images_dir = '/content/pill_data_v3/raw_data/test_images'
    print(f"\n'{test_images_dir}' 폴더의 이미지에 대한 예측을 시작합니다...")
    results = model.predict(source=test_images_dir, stream=True)
    print("✅ 예측이 완료되었습니다.")

    # 제출 파일 생성
    print("\n제출 파일 생성을 시작합니다...")
    submission_data = []
    annotation_id_counter = 1

    for result in tqdm(results, desc="파일 생성 중"):
        file_name = os.path.basename(result.path)
        image_id_match = re.search(r'\d+', file_name)
        if not image_id_match: continue
        image_id = int(image_id_match.group())

        for box in result.boxes:
            x_center, y_center, width, height = box.xywh[0].tolist()
            x_min, y_min = x_center - width / 2, y_center - height / 2

            # YAML 파일 기준의 정확한 ID 복원 로직
            yolo_class_id = int(box.cls[0])
            class_name = yolo_class_names[yolo_class_id]
            original_category_id = name_to_original_id_map.get(class_name, -1)

            score = float(box.conf[0])

            submission_data.append([
                annotation_id_counter, image_id, original_category_id,
                round(x_min, 3), round(y_min, 3), round(width, 3), round(height, 3),
                round(score, 3)
            ])
            annotation_id_counter += 1

    submission_df = pd.DataFrame(submission_data,
                                 columns=['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score'])

    submission_df.to_csv('submission_finetuned_v3_cuAG.csv', index=False)

    print("\n✅ 'submission_finetuned_v3_cuAG.csv' 파일 생성이 완료되었습니다.")
    display(submission_df.head())

else:
    print("❗️'categories_df'가 없습니다. 새로운 v3 데이터 통합 코드를 먼저 실행하여 변수를 생성해주세요.")

✅ 훈련에 사용된 YAML 파일에서 클래스 이름 목록을 성공적으로 불러왔습니다.
✅ 클래스 이름 -> 원본 ID 맵 생성이 완료되었습니다.
✅ 새로운 챔피언 모델 '/content/runs/detect/yolov8s_finetuned_on_v3_cuAG/weights/best.pt'을(를) 성공적으로 불러왔습니다.

'/content/pill_data_v3/raw_data/test_images' 폴더의 이미지에 대한 예측을 시작합니다...
✅ 예측이 완료되었습니다.

제출 파일 생성을 시작합니다...


파일 생성 중: 0it [00:00, ?it/s]


image 1/843 /content/pill_data_v3/raw_data/test_images/1.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 놀텍정 10mg, 1 울트라셋이알서방정, 36.3ms


파일 생성 중: 1it [00:00,  2.79it/s]

image 2/843 /content/pill_data_v3/raw_data/test_images/10.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 17.8ms


...

파일 생성 중: 842it [00:59, 15.23it/s]

image 843/843 /content/pill_data_v3/raw_data/test_images/999.png: 640x512 1 보령부스파정 5mg, 1 동아가바펜틴정 800mg, 1 라비에트정 20mg, 1 레일라정, 12.4ms
Speed: 2.6ms preprocess, 12.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 512)


파일 생성 중: 843it [00:59, 14.27it/s]



✅ 'submission_finetuned_v3_cuAG.csv' 파일 생성이 완료되었습니다.


annotation_id  image_id  category_id   bbox_x   bbox_y   bbox_w   bbox_h  \
0              1         1        27925  598.009  674.556  255.036  475.885   
1              2         1         1899  157.932  251.142  211.147  125.673   
2              3         1        16550  556.578   70.785  398.240  405.329   
3              4         1        24849  172.586  739.793  186.675  294.694   
4              5        10        16547  101.992  807.331  241.852  237.634   

   score  
0  0.962  
1  0.865  
2  0.827  
3  0.520  
4  0.974

In [ ]:
!zip -r runs.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/yolov8s_finetuned_on_v3/ (stored 0%)
  adding: runs/detect/yolov8s_finetuned_on_v3/args.yaml (deflated 53%)
  adding: runs/detect/yolov8s_finetuned_on_v3/BoxPR_curve.png (deflated 17%)
  adding: runs/detect/yolov8s_finetuned_on_v3/train_batch1.jpg (deflated 8%)
  adding: runs/detect/yolov8s_finetuned_on_v3/confusion_matrix_normalized.png (deflated 17%)
  adding: runs/detect/yolov8s_finetuned_on_v3/labels_correlogram.jpg (deflated 45%)
  adding: runs/detect/yolov8s_finetuned_on_v3/train_batch2.jpg (deflated 7%)
  adding: runs/detect/yolov8s_finetuned_on_v3/labels.jpg (deflated 31%)
  adding: runs/detect/yolov8s_finetuned_on_v3/BoxR_curve.png (deflated 9%)
  adding: runs/detect/yolov8s_finetuned_on_v3/BoxP_curve.png (deflated 8%)
  adding: runs/detect/yolov8s_finetuned_on_v3/val_batch0_pred.jpg (deflated 7%)
  adding: runs/detect/yolov8s_finetuned_on_v3/results.png (deflated 5%)
  adding: runs/detect/yol